In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm


import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# 1 Functions

## 1.1 Make Examples
1) Choose the frequency bins at which to calculate the GWB, same as in semi-analytic-models.ipynb
2) Build Semi-Analytic-Model with super simple parameters 
3) Get SAM edges and numbers as in sam.gwb()

In [ ]:
def example(dur, cad, mtot, mrat, redz, print_test):
    ''' 
    1) Choose the frequency bins at which to calculate the GWB, same as in semi-analytic-models.ipynb
    2) Build Semi-Analytic-Model with super simple parameters 
    3) Get SAM edges and numbers as in sam.gwb()

    Parameters
    ----------oiuyuiuytd
    dur : scalar
        Duration of observation in secnods (multiply by YR)
    cad : scalar
        Cadence of observations in seconds (multiply by YR)
    mtot : (3,) list of scalars
        Min, max, and steps for total mass.
    mrat : (3,) list of scalars
        Min, max, and steps for mass ratio.
    redz : (3,) list of scalars
        Min, max, and steps for redshift.
    print_test :

    Returns
    -------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M+1, Q+1, Z+1, F+1.
    number : (M, Q, Z, F) array
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    fobs : (F) array
        observed frequency bin centers
    '''
    # 1) Choose the frequency bins at which to calculate the GWB, same as in semi-analytic-models.ipynb
    fobs = utils.nyquist_freqs(dur,cad)
    fobs_edges = utils.nyquist_freqs_edges(dur,cad)
    if(print_test):
        print(f"Number of frequency bins: {fobs.size-1}")
        print(f"  between [{fobs[0]*YR:.2f}, {fobs[-1]*YR:.2f}] 1/yr")
        print(f"          [{fobs[0]*1e9:.2f}, {fobs[-1]*1e9:.2f}] nHz")

    # 2) Build Semi-Analytic-Model with super simple parameters 
    if(mtot==None or mrat==None or redz==None):
        print('using default mtot, mrat, and redz')
        sam = holo.sam.Semi_Analytic_Model()
    else:
        sam = holo.sam.Semi_Analytic_Model(mtot, mrat, redz)
    if(print_test):
        print('edges:', sam.edges)
    # get observed orbital frequency bin edges and centers 
    # from observed GW frequency bin edges
    fobs_orb_edges = fobs_edges / 2.0 # f_orb = f_GW/2
    fobs_orb_cents = kale.utils.midpoints(fobs_edges) / 2.0

    # 3) Get SAM edges and numbers as in sam.gwb()
    # dynamic_binary_number
    # gets differential number of binaries per bin-vol per log freq interval
    edges, dnum = sam.dynamic_binary_number(holo.hardening.Hard_GW, fobs_orb=fobs_orb_cents)
    edges[-1] = fobs_orb_edges

    # integrate (multiply by bin volume) within each bin
    number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
    number = number * np.diff(np.log(fobs_edges))

    return edges, number, fobs

Example 2

In [ ]:
def example2(print_test = True, exname='Example 2'):
    ''' 
    Parameters
    ---------
    print_test : Bool
        Whether to print frequencies and edges


    Returns
    ---------
    edges : (M,Q,Z,F) array
    number : (M, Q, Z, F) array
    fobs : (F) array
        observed frequency bin centers
    '''
    
    dur = 5.0*YR/3.1557600
    cad = .5*YR/3.1557600
    
    mtot=(1.0e6*MSOL/1.988409870698051, 1.0e8*MSOL/1.988409870698051, 3)
    mrat=(1e-1, 1.0, 2)
    redz=(1e-3, 1.0, 4)
    
    edges, number, fobs = example(dur, cad, mtot, mrat, redz, print_test)
    return edges, number, fobs, exname

Example 3

In [ ]:
def example3(print_test = True, exname = 'Example 3'):
    ''' 
    Parameters
    ---------
    print_test : Bool
        Whether to print frequencies and edges


    Returns
    ---------
    edges : (M,Q,Z,F) array
    number : (M, Q, Z, F) array
    fobs : (F) array
        observed frequency bin centers
    '''
    dur = 5.0*YR/3.1557600
    cad = .5*YR/3.1557600
    

    mtot=(1.0e6*MSOL/1.988409870698051, 4.0e9*MSOL, 25)
    mrat=(1e-1, 1.0, 25)
    redz=(1e-3, 10.0, 25)

    edges, number, fobs = example(dur, cad, mtot, mrat, redz, print_test)
    return edges, number, fobs, exname

Example 4

In [ ]:
def example4(print_test = True, exname = 'Example 4'):
    ''' 
    Parameters
    ---------
    print_test : Bool
        Whether to print frequencies and edges


    Returns
    ---------
    edges : (M,Q,Z,F) array
    number : (M, Q, Z, F) array
    fobs : (F) array
        observed frequency bin centers
    '''
    dur = 5.0*YR/3.1557600
    cad = .2*YR/3.1557600

    mtot=(1.0e6*MSOL/1.988409870698051, (4.0e11*MSOL).astype(np.float64), 25)
    mrat=(1e-1, 1.0, 25)
    redz=(1e-3, 10.0, 25)
    
    edges, number, fobs = example(dur, cad, mtot, mrat, redz, print_test)
    return edges, number, fobs, exname

Example 5 (same as in semi-analytic-models.ipynb)

In [ ]:
def example5(print_test = True, exname = 'Example 5'):
    ''' 
    Parameters
    ---------
    print_test : Bool
        Whether to print frequencies and edges


    Returns
    ---------
    edges : (M,Q,Z,F) array
    number : (M, Q, Z, F) array
    fobs : (F) array
        observed frequency bin centers
    '''
    dur = 10.0*YR
    cad = .2*YR

    # default mtot, mrat, redz
    
    edges, number, fobs = example(dur, cad, mtot=None, mrat=None, redz=None, 
                                  print_test=print_test)
    return edges, number, fobs, exname

## 1.2 SS Calculations
Contains: 
- ss_by_loops
- gws_by_ndars (same purpose as gravwaves._gws_from_number_grid_integrated())
- ss_by_ndars
- subtraction_from_number method (no longer needed)
Could add:
- get sspar from edges and ssidx (instead of calculating and returning sspar)
- get bgnum from number and ssidx (instead of ss methods returning bgnum)


old_ss_by_loops

In [ ]:
def old_ss_by_loops(edges, number, realize=False, round=True,  print_test = False):
       
    """ Inefficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M, Q, Z, F.
    number : (M, Q, Z, F) ndarray
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int,
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If a `int` value, then how many discrete realizations to construct.
    round : bool
        Specification of whether to discretize the sample if realize is False, 
        by rounding number of binaries in each bin to integers. 
    print_test : bool
        Whether or not to print variable as they are calculated, for dev purposes.


    Returns
    -------
    hc_bg : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False: shape is (F)
        realize = R, True: shape is  (F, R)
    hc_ss : (F) array
        The characteristic strain of the loudest single source at each frequency.
    sspar : (F, 3) 2darray or None
        The parameters (M, q, and z) of the loudest single source at each frequency.
    ssidx : (F, 3) 2darray or None
        The indices (m_idx, q_idx, and z_idx) of the parameters of the loudest single
        source's bin, at each frequency.    
    maxhs : (F) array or None
        The maximum single source strain amplitude at each frequency.
    bgnum : (M, Q, Z, F) 
        The number of binaries in each bin after the loudest single source
        at each frequency is subtracted out.

    """
    if(print_test):
        print('INPUTS: edges:', len(edges), '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift

    
    # GW background characteristic strain
    hc_bg = np.empty_like(number)

    # new number array
    if(round == True):
        bgnum = np.copy(np.floor(number).astype(np.int64))
        if(print_test):
            print('noninteger bgnum values:', bgnum[(bgnum%1 !=0)])
    else:
        bgnum = np.copy(number)


    # for single sources, make a grid with shape
    # (f, 3)
    # params of loudest bin with number>=1
    # shape (f,3) for 3 params
    sspar = np.empty((len(fc), 3)) 
    # param indices of loudest bin with number>=1
    # shape (f,3) for 3 params
    ssidx = np.empty((len(fc), 3)) 
    # max hs at each frequency
    maxhs = np.zeros(len(fc))
    # (max)  single source characteristic strain at each frequency
    hc_ss = np.zeros(len(fc))

    # # not worrying about realization implementation yet
    # # for r realizations, make a grid with shape 
    # # m, q, z, f, r
    # if(utils.isinteger(realize)):
    #     newshape = hc_grid.shape + (realize,)
    #     if(print_test):
    #         print('newshape:', newshape)
    #     realized_grid = np.empty(newshape)


    # --------------- Single Sources ------------------
    # 0) Round or realize so numbers are all integers
    # 1) Identify the loudest (max hs) single source in a bin with N>0 
    # 2) Record the parameters, parameter indices, and strain
    #  of that single source
    # 3) Subtract 1 from the number in that source's bin, 
    # 4) Calculate single source characteristic strain (hc)
    # 5) Calculate the background with the new number 
    
    for m_idx in range(len(mt)):
        for q_idx in range(len(mr)):
            cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
            for z_idx in range(len(rz)):
                cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                
                # print M, q, z, M_c, d_c
                if(print_test):
                    print('BIN mt=%.2e, mr=%.2e, rz=%.2e' %
                        (mt[m_idx], mr[q_idx], rz[z_idx]))
                    print('\t m_c = %.2e, d_c = %.2e' 
                        % (cmass, cdist))

                # check if loudest source in any bin
                for f_idx in range(len(fc)):
                    rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                    # hs of a source in that bin
                    hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                    
                    # 1) IF LOUDEST
                    # check if loudest hs at that 
                    # frequency and contains binaries
                    if(hs_mqzf>maxhs[f_idx] and 
                        bgnum[m_idx, q_idx, z_idx, f_idx]>0):
                        if(bgnum[m_idx, q_idx, z_idx, f_idx]<1):
                            print('number<1 used', bgnum[m_idx, q_idx, z_idx, f_idx])  #DELETE
                        # 2) If so, RECORD:
                        # parameters M, q, z
                        sspar[f_idx] = np.array([mt[m_idx], mr[q_idx],
                                                    rz[z_idx]])
                        # parameter indices
                        ssidx[f_idx] = np.array([m_idx, q_idx, z_idx])
                        # new max strain
                        maxhs[f_idx] = hs_mqzf


    # 3) SUBTRACT 1 
    # from bin with loudest source at each frequency
    # can do this using the index of loudest, ssidx
    # recall ssidx has shape [3, F]
    # and = [(m_idx,q_idx,z_idx), fc],
    for f_idx in range(len(fc)):
        bgnum[int(ssidx[f_idx,0]), int(ssidx[f_idx,1]), int(ssidx[f_idx,2]), 
                f_idx] -= 1 

        # 4) CALCULATE 
        # single source characteristic strain
        hc_ss[f_idx] = np.sqrt(maxhs[f_idx]**2 * (fc[f_idx]/df[f_idx]))
    
    # CHECK no numbers should be <0 
    if(np.any(bgnum<0)): 
        error_index = np.where(bgnum<0)
        print('number<0 found at (M,q,z,f) =', error_index)         
        

    # 5)
    # ----------------- Calculate Background Strains --------------------
    # then we can go back in and calculate characteristic strains
    for m_idx in range(len(mt)):
        for q_idx in range(len(mr)):
            cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
            for z_idx in range(len(rz)):
                cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                for f_idx in range(len(fc)):
                    rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                    hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                    hc_dlnf = hs_mqzf**2 * (fc[f_idx]/df[f_idx])
                    if(realize == False):
                        hc_bg[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                                        * bgnum[m_idx, q_idx, z_idx, f_idx])
                    else: 
                        raise Exception('realize not implemented yet') 
                    # elif(realize == True):
                    #     hc_grid[m_idx, q_idx, z_idx, f_idx] = np.sqrt(hc_dlnf 
                    #                     *np.random.poisson(bgnum[m_idx, q_idx, z_idx, f_idx]))
                    # elif(utils.isinteger(realize)):
                    #     for r_idx in range(reals):
                    #         realized_grid[m_idx, q_idx, z_idx, f_idx, r_idx] = \
                    #             np.sqrt(hc_dlnf 
                    #                     *np.random.poisson(bgnum[m_idx, q_idx, z_idx, f_idx]))
                    #     hc_grid = realized_grid                
                    # else:
                    #     print("`realize` ({}) must be one of {{True, False, integer}}!"\
                    #         .format(realize))

                    if(print_test):
                        print('\tfr = %.2fnHz, h_s = %.2e, h_c^2/dlnf = %.2e'
                            % (rfreq*10**9, hs_mqzf, hc_dlnf))
                        print('\t\tnumber: %.2e' % bgnum[m_idx, q_idx, z_idx, f_idx])
                        print('\t\thc = %.2e' % hc_bg[m_idx, q_idx, z_idx, f_idx])
                        print('\t\t loudest?', 
                            np.all((ssidx[f_idx] == np.array([m_idx, q_idx, z_idx]))))
    if(print_test):
        print('----loudest bins:')
        for f_idx in range(len(fc)):
            print('\t M=%.2e, q=%.2e, z=%.2e, f=%.2e' 
                    % (sspar[f_idx,0], sspar[f_idx,1], sspar[f_idx,2], fc[f_idx]))
            # if we use this instead, we don't need sspar
            # print('\t M=%.2e, q=%.2e, z=%.2e, f=%.2e' 
            #       % (mt[ssidx[f_idx,0]], mr[ssidx[f_idx,1]], rz[ssidx[f_idx,2]]))

     
    # sum over all bins at a given frequency and realization
    hc_bg = np.sqrt(np.sum(hc_bg**2, axis=(0, 1, 2)))

    
    return hc_bg, hc_ss, sspar, ssidx, maxhs, bgnum


ss_by_loops (with realizations)

In [ ]:
def ss_by_loops(edges, number, realize=False, round=True,  print_test = False):
       
    """ Inefficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M, Q, Z, F.
    number : (M, Q, Z, F) ndarray
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int,
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If a `int` value, then how many discrete realizations to construct.
    round : bool
        Specification of whether to discretize the sample if realize is False, 
        by rounding number of binaries in each bin to integers. 
    print_test : bool
        Whether or not to print variable as they are calculated, for dev purposes.


    Returns
    -------
    hc_bg : (F, R) ndarray
        Characteristic strain of the GWB.
        R=1 if realize is True or False, 
        R=realize if realize is an integer
    hc_ss : (F, R) ndarray
        The characteristic strain of the loudest single source at each frequency,
        for each realization. (R=1 if realize is True or False, otherwise R=realize)
    sspar : (F, 3, R) ndarray 
        The parameters (M, q, and z) of the loudest single source at each frequency.
    ssidx : (F, 3, R) ndarray 
        The indices (m_idx, q_idx, and z_idx) of the parameters of the loudest single
        source's bin, at each frequency.    
    maxhs : (F, R) ndarray 
        The maximum single source strain amplitude at each frequency.
    bgnum : (M, Q, Z, F, R) 
        The number of binaries in each bin after the loudest single source
        at each frequency is subtracted out. 
        R=1 if realize is True or False, 
        R=realize if realize is an integer

    """
    if(print_test):
        print('INPUTS: edges:', len(edges), '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift

    

    # new number array
    if(realize == False):
        reals = 1
        if(round == True):
            bgnum = np.copy(np.floor(number).astype(np.int64))[...,np.newaxis]
            if(print_test):
                print('noninteger bgnum values:', bgnum[(bgnum%1 !=0)])
        else:
            bgnum = np.copy(number)[...,np.newaxis]
    elif(realize == True):
        reals = 1
        bgnum = np.empty(number.shape + (reals,))
    elif(utils.isinteger(realize)):
        reals = realize
        bgnum = np.empty(number.shape + (reals,))
    else: 
        raise Exception("`realize` ({}) must be one of {{True, False, integer}}!"\
                            .format(realize))

    # GW background characteristic strain
    hc_bg = np.empty_like(bgnum, dtype=float)

    # for single sources, make a grid with shape
    # (f, 3)
    # params of loudest bin with number>=1
    # shape (f,3,R) for 3 params
    sspar = np.empty((len(fc), 3, reals)) 
    # param indices of loudest bin with number>=1
    # shape (f,3,R) for 3 params
    ssidx = np.empty((len(fc), 3, reals)) 
    # max hs at each frequency
    maxhs = np.zeros((len(fc), reals))
    # (max)  single source characteristic strain at each frequency
    hc_ss = np.zeros((len(fc), reals))


    # --------------- Single Sources ------------------
    # 0) Round or realize so numbers are all integers
    # 1) Identify the loudest (max hs) single source in a bin with N>0 
    # 2) Record the parameters, parameter indices, and strain
    #  of that single source
    # 3) Subtract 1 from the number in that source's bin, 
    # 4) Calculate single source characteristic strain (hc)
    # 5) Calculate the background with the new number 
    
    for m_idx in range(len(mt)):
        for q_idx in range(len(mr)):
            cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
            for z_idx in range(len(rz)):
                cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                
                # print M, q, z, M_c, d_c
                if(print_test):
                    print('BIN mt=%.2e, mr=%.2e, rz=%.2e' %
                        (mt[m_idx], mr[q_idx], rz[z_idx]))
                    print('\t m_c = %.2e, d_c = %.2e' 
                        % (cmass, cdist))

                # check if loudest source in any bin
                for f_idx in range(len(fc)):
                    rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                    # hs of a source in that bin
                    hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                    
                    for r_idx in range(reals):
                        # Poisson sample to get number in that bin if realizing 
                        # (otherwise it was already set above by copying or rounding.)
                        if(utils.isinteger(realize) or realize==True):
                            bgnum[m_idx, q_idx, z_idx, f_idx, r_idx] \
                                = np.random.poisson(number[m_idx,q_idx,z_idx,f_idx])
                        # 1) IF LOUDEST
                        # check if loudest hs at that 
                        # frequency and contains binaries

                        if(hs_mqzf>maxhs[f_idx, r_idx] and 
                            bgnum[m_idx, q_idx, z_idx, f_idx, r_idx]>0):
                            if(bgnum[m_idx, q_idx, z_idx, f_idx, r_idx]<1):
                                print('number<1 used', bgnum[m_idx, q_idx, z_idx, f_idx, r_idx])  #DELETE
                            # 2) If so, RECORD:
                            # parameters M, q, z
                            sspar[f_idx,:,r_idx] = np.array([mt[m_idx], mr[q_idx],
                                                        rz[z_idx]])
                            # parameter indices
                            ssidx[f_idx,:,r_idx] = np.array([m_idx, q_idx, z_idx])
                            # new max strain
                            maxhs[f_idx,r_idx] = hs_mqzf


    # 3) SUBTRACT 1 
    # from bin with loudest source at each frequency
    # can do this using the index of loudest, ssidx
    # recall ssidx has shape [3, F]
    # and = [(m_idx,q_idx,z_idx), fc],
    for f_idx in range(len(fc)):
        for r_idx in range(reals):
            bgnum[int(ssidx[f_idx,0,r_idx]), int(ssidx[f_idx,1,r_idx]), int(ssidx[f_idx,2,r_idx]), 
                f_idx, r_idx] -= 1 

            # 4) CALCULATE 
            # single source characteristic strain
            hc_ss[f_idx, r_idx] = np.sqrt(maxhs[f_idx, r_idx]**2 * (fc[f_idx]/df[f_idx]))
    
    # CHECK no numbers should be <0 
    if(np.any(bgnum<0)): 
        error_index = np.where(bgnum<0)
        print('number<0 found at (M,q,z,f) =', error_index)         
        

    # 5)
    # ----------------- Calculate Background Strains --------------------
    # then we can go back in and calculate characteristic strains
    # NOTE: could make this faster by saving rfreq and hs values from above
    # instead of recalculating
    for m_idx in range(len(mt)):
        for q_idx in range(len(mr)):
            cmass = holo.utils.chirp_mass_mtmr(mt[m_idx], mr[q_idx])
            for z_idx in range(len(rz)):
                cdist = holo.cosmo.comoving_distance(rz[z_idx]).cgs.value
                for f_idx in range(len(fc)):
                    rfreq = holo.utils.frst_from_fobs(fc[f_idx], rz[z_idx])
                    hs_mqzf = utils.gw_strain_source(cmass, cdist, rfreq)
                    hc_dlnf = hs_mqzf**2 * (fc[f_idx]/df[f_idx])
                    for r_idx in range(reals):
                        hc_bg[m_idx, q_idx, z_idx, f_idx, r_idx] = np.sqrt(hc_dlnf 
                                        * bgnum[m_idx, q_idx, z_idx, f_idx, r_idx])

    # sum over all bins at a given frequency and realization
    hc_bg = np.sqrt(np.sum(hc_bg**2, axis=(0, 1, 2)))

    
    return hc_bg, hc_ss, sspar, ssidx, maxhs, bgnum


gws_by_ndars (no ss)

In [ ]:
def gws_by_ndars(edges, number, realize, round = True, sum = True, print_test = False):
       
    """ More efficient way to calculate strain from numbered 
    grid integrated

    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions 
        correspond to total mass, mass ratio, redshift, and observer-frame orbital 
        frequency. The length of each of the four arrays is M, Q, Z, F.
    number : (M, Q, Z, F) ndarray
        The number of binaries in each bin of parameter space.  This is calculated 
        by integrating `dnum` over each bin.
    realize : bool or int
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If an `int` value, then how many discrete realizations to construct.
    round : bool
        Specification of whether to discretize the sample if realize is False, 
        by rounding number of binaries in each bin to integers. This has no impact 
        if realize is true.
        NOTE: should add a warning if round and realize are both True
    sum : bool
        Whether or not to sum the strain at a given frequency over all bins.
    print_test : bool
        Whether or not to print variable as they are calculated, for dev purposes.


    Returns
    -------
    hchar : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False, sum = False: shape is (M, Q, Z, F)
        realize = True or False, sum = True: shape is (F)
        realize = R, sum = False: shape is  (M, Q, Z, F, R)
        realize = R, sum = True: shape is  (F, R)

    """

    if(print_test):
        print('INPUTS: edges:', len(edges), # '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift


    # --- Chirp Masses ---
    # to get chirp mass in shape (M, Q) we need 
    # mt in shape (M, 1) 
    # mr in shape (1, Q)
    cmass = utils.chirp_mass_mtmr(mt[:,np.newaxis], mr[np.newaxis,:])
    if(print_test):
        print('cmass:', cmass.shape, '\n', cmass)

    # --- Comoving Distances ---
    # to get cdist in shape (Z) we need
    # rz in shape (Z)
    cdist = holo.cosmo.comoving_distance(rz).cgs.value
    if(print_test):
        print('cdist:', cdist.shape, '\n', cdist)

    # --- Rest Frame Frequencies ---
    # to get rest freqs in shape (Z, F) we need 
    # rz in shape (Z, 1) 
    # fc in shape (1, F)
    rfreq = holo.utils.frst_from_fobs(fc[np.newaxis,:], rz[:,np.newaxis])
    if(print_test):
        print('rfreq:', rfreq.shape, '\n', rfreq)

    # --- Source Strain Amplitude ---
    # to get hs amplitude in shape (M, Q, Z, F) we need
    # cmass in shape (M, Q, 1, 1) from (M, Q)
    # cdist in shape (1, 1, Z, 1) from (Z)
    # rfreq in shape (1, 1, Z, F) from (Z, F)
    hsamp = utils.gw_strain_source(cmass[:,:,np.newaxis,np.newaxis],
                                   cdist[np.newaxis,np.newaxis,:,np.newaxis],
                                   rfreq[np.newaxis,np.newaxis,:,:])
    if(print_test):
        print('hsamp', hsamp.shape, '\n', hsamp)

    # --- Characteristic Strain Squared ---
    # to get characteristic strain in shape (M, Q, Z, F) we need
    # hsamp in shape (M, Q, Z, F)
    # fc in shape (1, 1, 1, F)
    hchar = hsamp**2 * (fc[np.newaxis, np.newaxis, np.newaxis,:]
                        /df[np.newaxis, np.newaxis, np.newaxis,:])

    # Sample:
    if(realize == False):
        # without sampling, want strain in shape (M, Q, Z, F)
        if(round): 
            # discretize by rounding number down to nearest integer 
            hchar *= np.floor(number).astype(int) 
        else: 
            # keep non-integer values
            hchar *= number

    if(realize == True):
        # with a single sample, want strain in shape (M, Q, Z, F)
        hchar *= np.random.poisson(number)

    if(utils.isinteger(realize)):
        # with R realizations, 
        # to get strain in shape (M, Q, Z, F, R) we need
        # hchar in shape(M, Q, Z, F, 1)
        # Poisson sample in shape (1, 1, 1, 1, R)
        npois = np.random.poisson(number[...,np.newaxis], size = (number.shape + (realize,)))
        if(print_test):
            print('npois', npois.shape)
        hchar = hchar[...,np.newaxis] * npois


    if(print_test):
        print('hchar', hchar.shape, '\n', hchar)


    if(sum):
        # sum over all bins at a given frequency and realization
        hchar = np.sum(hchar, axis=(0, 1, 2))
        if(print_test):
            print('hchar summed', hchar.shape, '\n', hchar)

    return np.sqrt(hchar)

old_ss_by_ndars

In [ ]:
def old_ss_by_ndars(edges, number, realize, round = True, print_test = False):
       
    """ More efficient way to calculate strain from numbered 
    grid integrated


    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M, Q, Z, F.
    number : (M, Q, Z, F) ndarray of scalars
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int,
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If a `int` value, then how many discrete realizations to construct.
    round : bool
        Specification of whether to discretize the sample if realize is False, 
        by rounding number of binaries in each bin to integers. 
        Does nothing if realize is True.
    ss : bool 
        Whether or not to separate the loudest single source in each frequency bin.
    sum : bool
        Whether or not to sum the strain at a given frequency over all bins.
    print_test : bool
        Whether or not to print variable as they are calculated, for dev purposes.


    Returns
    -------
    hc_bg : ndarray
        Characteristic strain of the GWB.
        The shape depends on whether realize is an integer or not
        realize = True or False: shape is (F)
        realize = R: shape is  (F, R)
    hc_ss : (F,) array of scalars
        The characteristic strain of the loudest single source at each frequency.
    ssidx : (F, 4) ndarray 
        The indices (m_idx, q_idx, z_idx, f_idx) of the parameters of the loudest single
        source's bin, at each frequency such that 
        ssidx[i,0] = m_idx of the ith frequency
        ssidx[i,1] = q_idx of the ith frequency
        ssidx[i,2] = z_idx of the ith frequency
        ssidx[i,3] = f_idx of the ith frequency = i
    hsmax : (F) array of scalars 
        The maximum single source strain amplitude at each frequency.
    bgnum : (M, Q, Z, F) ndarray
        The number of binaries in each bin after the loudest single source
        at each frequency is subtracted out.
    ssnew : (4, F) ndarray
        The indices of loudest single sources at each frequency in the
        format: [[M indices], [q indices], [z indices], [f indices]]

        

    Potential BUG: In the unlikely scenario that there are two equal hsmaxes 
    (at same OR dif frequencies), ssidx calculation will go wrong
    Could avoid this by using argwhere for each f_idx column separately.
    Or TODO implement some kind of check to see if any argwheres return multiple 
    values for that hsmax and raises a warning/assertion error


    TODO: Calculate sspar
    TODO: Implement realizations
    TODO: Implement not summing, or remove option
    """

    if(print_test):
        print('INPUTS: edges:', len(edges), # '\n', edges, 
        '\nINPUTS:number:', number.shape, '\n', number,'\n')

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift


    # --- Chirp Masses ---
    # to get chirp mass in shape (M, Q) we need 
    # mt in shape (M, 1) 
    # mr in shape (1, Q)
    cmass = utils.chirp_mass_mtmr(mt[:,np.newaxis], mr[np.newaxis,:])
    if(print_test):
        print('cmass:', cmass.shape, '\n', cmass)

    # --- Comoving Distances ---
    # to get cdist in shape (Z) we need
    # rz in shape (Z)
    cdist = holo.cosmo.comoving_distance(rz).cgs.value
    if(print_test):
        print('cdist:', cdist.shape, '\n', cdist)

    # --- Rest Frame Frequencies ---
    # to get rest freqs in shape (Z, F) we need 
    # rz in shape (Z, 1) 
    # fc in shape (1, F)
    rfreq = holo.utils.frst_from_fobs(fc[np.newaxis,:], rz[:,np.newaxis])
    if(print_test):
        print('rfreq:', rfreq.shape, '\n', rfreq)

    # --- Source Strain Amplitude ---
    # to get hs amplitude in shape (M, Q, Z, F) we need
    # cmass in shape (M, Q, 1, 1) from (M, Q)
    # cdist in shape (1, 1, Z, 1) from (Z)
    # rfreq in shape (1, 1, Z, F) from (Z, F)
    hsamp = utils.gw_strain_source(cmass[:,:,np.newaxis,np.newaxis],
                                   cdist[np.newaxis,np.newaxis,:,np.newaxis],
                                   rfreq[np.newaxis,np.newaxis,:,:])
    if(print_test):
        print('hsamp', hsamp.shape, '\n', hsamp)


    ############################################################
    ########## HERE'S WHERE THINGS CHANGE FOR SS ###############
    ############################################################

    # --------------- Single Sources ------------------
    ##### 0) Round and/or realize so numbers are all integers
    if (round == True):
        bgnum = np.copy(np.floor(number).astype(np.int64))
        assert (np.all(bgnum%1 == 0)), 'non integer numbers found with round=True'
        assert (np.all(bgnum >= 0)), 'negative numbers found with round=True'
    else:
        bgnum = np.copy(number)
        if(ss==True):
            warnings.warn('Number grid used for single source calculation.')

    if(realize == True):
        bgnum = np.random.poisson(number)
        assert (np.all(bgnum%1 ==0)), 'nonzero numbers found with realize=True'
    print('bgnum stats after copy\n', holo.utils.stats(bgnum))

    #### 1) Identify the loudest (max hs) single source in a bin with N>0 
    hsamp[(bgnum==0)] = 0 #set hs=0 if number=0
    # hsamp[bgnum==0] = 0



    # --- Single Source Strain Amplitude At Each Frequency ---
    # to get max strain in shape (F) we need
    # hsamp in shape (M, Q, Z, F), search over first 3 axes
    hsmax = np.amax(hsamp, axis=(0,1,2)) #find max hs at each frequency
    
    #### 2) Record the indices and strain of that single source

    # --- Indices of Loudest Bin ---
    # Shape (F, 4), looks like
    # [[m_idx,q_idx,z_idx,0],
    #  [m_idx,q_idx,z_idx,1],
    #   ........
    #  [m_idx,q_idx,z_idx,F-2]]
    # no longer actually need this, but might be useful
    ssidx = np.argwhere(hsamp==hsmax) 
    ssidx = ssidx[ssidx[:,-1].argsort()]

    # --- Indices of Loudest Bin, New Method ---
    # Shape (4, F), looks like
    # [[m1,m2,..mN], [q1,q2,...qN], [z1,z2,...zN], [0,1,...N-1]]
    # for N=F frequencies
    shape = hsamp.shape
    newshape = (shape[0]*shape[1]*shape[2], shape[3])
    hsamp = hsamp.reshape(newshape) # change hsamp to shape (M*Q*Z, F)
    argmax = np.argmax(hsamp, axis=0) # max at each frequency
    hsamp = hsamp.reshape(shape) # restore hsamp shape to (M, Q, Z, F)
    mqz = np.array(np.unravel_index(argmax, shape[:-1])) # unravel indices
    f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
    ssnew = np.append(mqz, f_ids).reshape(4,len(f_ids))



    ### 3) Subtract 1 from the number in that source's bin

    # --- Background Number ---
    # bgnum = subtract_from_number(bgnum, ssidx) # Find a better way to do this!
    if np.any( bgnum[(hsamp == hsmax)] <=0):
        raise Exception("bgnum <= found at hsmax")
    if np.any( hsamp[(hsamp == hsmax)] <=0):
        raise Exception("hsamp <=0 found at hsmax")
    if np.any(hsmax<=0):
        raise Exception("hsmax <=0 found")
   
    # print('bgnum stats:\n', holo.utils.stats(bgnum))
    # print('bgnum[hsamp==hsmax] stats:\n', holo.utils.stats(bgnum[(hsamp == hsmax)]))
    bgnum[(hsamp == hsmax)]-=1 
    # NOTE keep an eye out for if hsmax is not found anywhere in hsasmp
    # could change to bgnum(np.where(hsamp==hsmax) & (bgnum >0))-=1
    # print('\nafter subtraction')
    # print('bgnum stats:\n', holo.utils.stats(bgnum))
    # print('bgnum[hsamp==hsmax] stats:\n', holo.utils.stats(bgnum[(hsamp == hsmax)]))

    assert np.all(bgnum>=0), f"bgnum contains negative values at: {np.where(bgnum<0)}"
    # if(np.any(bgnum<0)):   # alternate way to check for this error, and give index of neg number
    #         error_index = *np.where(bgnum<0)
    #         print('number<0 found at [M's], [q's], [z's], [f's]) =', error_index)   

    

    ### 4) Calculate single source characteristic strain (hc)

    # --- Single Source Characteristic Strain ---
    # to get ss char strain in shape [F] need
    # fc in shape (F)
    # df in shape (F)
    hc_ss = np.sqrt(hsmax**2 * (fc/df))


    # --- Parameters of loudest source ---
    # NOTE: This would be useful to implement, or have separate function for

    ### 5) Calculate the background with the new number 
 
    # --- Background Characteristic Strain Squared ---
    # to get characteristic strain in shape (M, Q, Z, F) we need
    # hsamp in shape (M, Q, Z, F)
    # fc in shape (1, 1, 1, F)
    hchar = hsamp**2 * (fc[np.newaxis, np.newaxis, np.newaxis,:]
                        /df[np.newaxis, np.newaxis, np.newaxis,:])   
    if (realize==False):
        hchar *= bgnum
    else:
        raise Exception('realize not implemented yet') 
        

    if(print_test):
        print('hchar', hchar.shape, '\n', hchar)

    
    # sum over all bins at a given frequency
    hchar = np.sum(hchar, axis=(0, 1, 2))
    if(print_test):
        print('hchar summed', hchar.shape, '\n', hchar)

    hc_bg = np.sqrt(hchar)

    return hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum, ssnew


ss_by_ndars (with realizations)

In [ ]:
def ss_by_ndars(edges, number, realize, round = True):
       
    """ More efficient way to calculate strain from numbered 
    grid integrated


    Parameters
    ----------
    edges : (4,) list of 1darrays
        A list containing the edges along each dimension.  The four dimensions correspond to
        total mass, mass ratio, redshift, and observer-frame orbital frequency.
        The length of each of the four arrays is M+1, Q+1, Z+1, F+1.
    number : (M, Q, Z, F) ndarray of scalars
        The number of binaries in each bin of parameter space.  This is calculated by integrating
        `dnum` over each bin.
    realize : bool or int,
        Specification of how to construct one or more discrete realizations.
        If a `bool` value, then whether or not to construct a realization.
        If a `int` value, then how many discrete realizations to construct.
    round : bool
        Specification of whether to discretize the sample if realize is False, 
        by rounding number of binaries in each bin to integers. 
        Does nothing if realize is True.
    ss : bool 
        Whether or not to separate the loudest single source in each frequency bin.
    sum : bool
        Whether or not to sum the strain at a given frequency over all bins.
    print_test : bool
        Whether or not to print variable as they are calculated, for dev purposes.


    Returns
    -------
    hc_bg : (F, R) ndarray of scalars
        Characteristic strain of the GWB.
        If realize = True or False: R is 1
        If realize is an integer, R=realize
    hc_ss : (F, R) ndarray of scalars
        The characteristic strain of the loudest single source at each frequency.
    hsamp : (M, Q, Z, F, R) ndarray
        Strain amplitude of a single source in every bin (but equal to zero 
        if the number in that bin is 0)
    ssidx : (5, F, R) ndarray
        The indices of loudest single sources at each frequency of each realization
        in the format: [[M indices], [q indices], [z indices], [f indices], [r indices]] 
    hsmax : (F, R) array of scalars 
        The maximum single source strain amplitude at each frequency.
    bgnum : (M, Q, Z, F, R) ndarray
        The number of binaries in each bin after the loudest single source
        at each frequency is subtracted out.
    

    In the unlikely scenario that there are two equal hsmaxes 
    (at same OR dif frequencies), ssidx calculation will go wrong
    Could avoid this by using argwhere for each f_idx column separately.
    Or TODO implement some kind of check to see if any argwheres return multiple 
    values for that hsmax and raises a warning/assertion error


    TODO: Calculate sspar
    TODO: Implement realizations
    TODO: Implement not summing, or remove option
    """

    # Frequency bin midpoints
    foo = edges[-1]                   #: should be observer-frame orbital-frequencies
    df = np.diff(foo)                 #: frequency bin widths
    fc = kale.utils.midpoints(foo)    #: use frequency-bin centers for strain (more accurate!)
    F = len(fc)                       #: number of frequencies

    # All other bin midpoints
    mt = kale.utils.midpoints(edges[0]) #: total mass
    mr = kale.utils.midpoints(edges[1]) #: mass ratio
    rz = kale.utils.midpoints(edges[2]) #: redshift


    # --- Chirp Masses ---
    # to get chirp mass in shape (M, Q) we need 
    # mt in shape (M, 1) 
    # mr in shape (1, Q)
    cmass = utils.chirp_mass_mtmr(mt[:,np.newaxis], mr[np.newaxis,:])

    # --- Comoving Distances ---
    # to get cdist in shape (Z) we need
    # rz in shape (Z)
    cdist = holo.cosmo.comoving_distance(rz).cgs.value

    # --- Rest Frame Frequencies ---
    # to get rest freqs in shape (Z, F) we need 
    # rz in shape (Z, 1) 
    # fc in shape (1, F)
    rfreq = holo.utils.frst_from_fobs(fc[np.newaxis,:], rz[:,np.newaxis])

    # --- Source Strain Amplitude ---
    # to get hs amplitude in shape (M, Q, Z, F) we need
    # cmass in shape (M, Q, 1, 1) from (M, Q)
    # cdist in shape (1, 1, Z, 1) from (Z)
    # rfreq in shape (1, 1, Z, F) from (Z, F)
    hsamp = utils.gw_strain_source(cmass[:,:,np.newaxis,np.newaxis],
                                   cdist[np.newaxis,np.newaxis,:,np.newaxis],
                                   rfreq[np.newaxis,np.newaxis,:,:])



    ############################################################
    ########## HERE'S WHERE THINGS CHANGE FOR SS ###############
    ############################################################

    # --------------- Single Sources ------------------
    ##### 0) Round and/or realize so numbers are all integers

    # --- Background Number ---
    # in shape (M, Q, Z, F, R)
    if(realize == False):
        R=1
        if (round == True):
            bgnum = np.copy(np.floor(number).astype(np.int64))[...,np.newaxis]
            assert (np.all(bgnum%1 == 0)), 'non integer numbers found with round=True'
            assert (np.all(bgnum >= 0)), 'negative numbers found with round=True'
        else:
            bgnum = np.copy(number)[...,np.newaxis]
    elif(realize == True):
        R=1
        bgnum = np.random.poisson(number[...,np.newaxis], 
                                size = (number.shape + (R,)))
        assert (np.all(bgnum%1 ==0)), 'nonzero numbers found with realize=True'
    elif(utils.isinteger(realize)):
        R=realize
        bgnum = np.random.poisson(number[...,np.newaxis], 
                                size = (number.shape + (R,)))
    else:
        print("`realize` ({}) must be one of {{True, False, integer}}!"\
            .format(realize))   

    #### 1) Identify the loudest (max hs) single source in a bin with N>0 
    # NOTE this requires hsamp for every realization, might be slow/big
    # --- Bin Strain Amplitudes ---
    # to get hsamp in shape (M, Q, Z, F, R)
    hsamp = np.repeat(hsamp[...,np.newaxis], R, axis=4)
    hsamp[(bgnum==0)] = 0 #set hs=0 if number=0
    # hsamp[bgnum==0] = 0


    # --- Single Source Strain Amplitude At Each Frequency ---
    # to get max strain in shape (F) we need
    # hsamp in shape (M, Q, Z, F), search over first 3 axes
    hsmax = np.amax(hsamp, axis=(0,1,2)) #find max hs at each frequency
    

    #### 2) Record the indices and strain of that single source

    # --- Indices of Loudest Bin ---
    # Shape (5, F, R), 
    # where ssidx[:,F,R] = m, q, z, f, r indices for loudest source at F, R
    shape = hsamp.shape
    newshape = (shape[0]*shape[1]*shape[2], shape[3], shape[4])
    hsamp = hsamp.reshape(newshape) # change hsamp to shape (M*Q*Z, F, R)
    argmax = np.argmax(hsamp, axis=0) # max at each frequency
    hsamp = hsamp.reshape(shape) # restore hsamp shape to (M, Q, Z, F, R)
    mqz = np.array(np.unravel_index(argmax, shape[:-2]))         # shape (3, F, R)
    # add frequency indices
    f_ids = np.arange(len(mqz[0])).astype(int) # shape (F)
    f_ids = np.repeat(f_ids, R).reshape(F, R)[np.newaxis,...]    # shape (1, F, R)
    ssidx = np.append(mqz, f_ids, axis=0)                        # shape (4, F, R)
    # add realization indices
    r_ids = np.arange(R).astype(int)                     # shape (R)
    r_ids = np.tile(r_ids, F).reshape(F,R)[np.newaxis,...]       # shape (1, F, R)
    ssidx = np.append(ssidx, r_ids, axis=0)                      # shape (5, F, R)






    #### 3) Subtract 1 from the number in that source's bin

    # --- Background Number ---
    # bgnum = subtract_from_number(bgnum, ssidx) # Find a better way to do this!
    if np.any( bgnum[(hsamp == hsmax)] <=0):
        raise Exception("bgnum <= found at hsmax")
    if np.any( hsamp[(hsamp == hsmax)] <=0):
        raise Exception("hsamp <=0 found at hsmax")
    if np.any(hsmax<=0):
        raise Exception("hsmax <=0 found")
   
    # print('bgnum stats:\n', holo.utils.stats(bgnum))
    # print('bgnum[hsamp==hsmax] stats:\n', holo.utils.stats(bgnum[(hsamp == hsmax)]))
    # print('ssidx\n', ssidx, ssidx.shape)
    bgnum[ssidx[0], ssidx[1], ssidx[2], ssidx[3], ssidx[4]] -= 1
    # print('\nafter subtraction')
    # print('bgnum stats:\n', holo.utils.stats(bgnum))
    # print('bgnum[hsamp==hsmax] stats:\n', holo.utils.stats(bgnum[(hsamp == hsmax)]))

    assert np.all(bgnum>=0), f"bgnum contains negative values at: {np.where(bgnum<0)}"
    

    #### 4) Calculate single source characteristic strain (hc)

    # --- Single Source Characteristic Strain ---
    # to get ss char strain in shape [F,R] need
    # fc in shape (F,1)
    # df in shape (F,1)
    # hsmax in shape (F,R)
    hc_ss = np.sqrt(hsmax**2 * (fc[:,np.newaxis]/df[:,np.newaxis]))


    
    #### 5) Calculate the background with the new number 
 
    # --- Background Characteristic Strain Squared ---
    # to get characteristic strain in shape (M, Q, Z, F, R) we need
    # hsamp in shape (M, Q, Z, F, R)
    # fc in shape (1, 1, 1, F, 1)
    hchar = hsamp**2 * (fc[np.newaxis, np.newaxis, np.newaxis,:,np.newaxis]
                        /df[np.newaxis, np.newaxis, np.newaxis,:,np.newaxis])   
    hchar *= bgnum

        

    # --- Background Characteristic Strain  ---
    # hc_bg in shape (F, R)
    hchar = np.sum(hchar, axis=(0, 1, 2)) # sum over all bins at a given frequency
    hc_bg = np.sqrt(hchar) # sqrt

    return hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum


## 1.3 Test

plotting

In [ ]:
def add_sampleGWB_to_plot(fig, ax, PLOT_GWB, LABEL, COLOR='b'):
    nsamp = PLOT_GWB.shape[1]
    ax.plot(xx, np.median(PLOT_GWB, axis=1), color=COLOR, label=LABEL)
    # plot contours at 50% and 98% confidence intervals
    for pp in [50, 98]:
        percs = pp / 2
        percs = [50 - percs, 50 + percs]
        ax.fill_between(xx, *np.percentile(PLOT_GWB, percs, axis=-1), alpha=0.25, color=COLOR)
    # Plot `nsamp` random spectra 
    
    idx = np.random.choice(PLOT_GWB.shape[1], nsamp, replace=False)
    ax.plot(xx, PLOT_GWB[:, idx], lw=1.0, alpha=0.5, color=COLOR, linestyle = 'dotted')

test functions

In [ ]:
def max_test(hsmax, hsamp): 
    # check hsmaxes are correct
    hsmax_hsamp_match = np.empty_like(hsmax)
    for f_idx in range(len(hsmax)):
        for r_idx in range(len(hsmax[0])):
            hsmax_hsamp_match[f_idx] = (np.max(hsamp[...,f_idx, r_idx]) == hsmax[f_idx, r_idx])
    assert np.all(hsmax_hsamp_match == True), "the max amplitudes in hsamp do not match those in hsmax"
    print('max_test passed')

def ssidx_test(hsmax, hsamp, ssidx, print_test):
    """ 
    Test ssidx in hsamp gives the same values as hsmax

    Parameters
    ----------
    hsmax : (F,) array of scalars
        Maximum strain amplitude of a single source at each frequency.
    hsamp : (M, Q, Z, F,) ndarray of scalar
        Strain amplitude of a source in each bin
    ssidx : (F, 4) ndarray 
        

    """
    # check ssidx are correct and in frequency order
    for ff in range(len(hsmax)): #ith frequency
        for rr in range(len(hsmax[0])):
            m,q,z,f,r = ssidx[:,ff,rr]
            if(print_test):
                print('max is at m,q,z,f,r = %d, %d, %d, %d, %d and it = %.2e'
                  % (m, q, z, f, r, hsmax[ff,rr]))
            assert (hsamp[m,q,z,f,r] == hsmax[ff,rr]), f"The ssidx[{ff},{rr}] does not give the hsmax[{ff},{rr}]."
    print('ssidx test passes')


def ssnew_test(hsmax, hsamp, ssnew, print_test):
    """ 
    Test ssnew in hsamp gives the same values as hsmax

    Parameters
    ----------
    hsmax : (F,) array of scalars
        Maximum strain amplitude of a single source at each frequency.
    hsamp : (M, Q, Z, F,) ndarray of scalar
        Strain amplitude of a source in each bin
    ssnew : (4, F) ndarray 
        

    """
    maxes = hsamp[ssnew[0], ssnew[1], ssnew[2], ssnew[3]]
    if(print_test):
        print('maxes by hsamp[ssnew[0], ssnew[1], ssnew[2], ssnew[3]] are:',
        maxes)
    
    assert np.all(maxes == hsmax), f"ssnew does not give correct hs maxes"
    print('ssnew test passes')



def number_test(num, bgnum, fobs, exname='', plot_test=False):
    ''' 
    Plots num - bgnum, where number is the ndarray of 
    integer number of sources in each bin, i.e. after 
    rounding or Poisson sampling

    Parameters
    ------------
    num : (M, Q, Z, F) array
        integer numbers in each bin, i.e. after rounding or
        Poisson sampling
    bgnum : (M, Q, Z, F) array
        number of background sources in each bin, 
        after single source subtraction
    fobs : (F) array
        frequencies of each F, for ax titles
    exname : String
        name of example
    plot_test : Bool
        whether or not to print values a


    Returns
    -----------
    None 
    
    '''   
    if np.all(num%1 == 0) != True: warnings.warn("num contains at least one non-integer value")
    difs = num - bgnum
    assert len(difs[np.where(difs>0)]) == len(difs[0,0,0,:]), "More than one bin per frequency found with a single source subtracted."

    if(plot_test):
        fig, ax = plt.subplots(1,len(fobs), figsize = (10,3), sharey=True)
        fig.suptitle('integer number - numbg for each bin, '+ exname)
        ax[0].set_ylabel('number - number_bg')
        bins = np.arange(0, num[...,0].size, 1)
        bins = np.reshape(bins, num[...,0].shape)
        # print(bins.shape)
        # print(num[...,0].shape)
        for f in range(len(fobs)):
            ax[f].scatter(bins, (num[...,f] - bgnum[...,f]))
            ax[f].set_title('$f_\mathrm{obs}$ = %dnHz' % (fobs[f]*10**9))
            ax[f].set_xlabel('bin')
        fig.tight_layout()
    print('number test passed')

    
    # TODO Assertion 

def compare_to_loops_test(edges, number, hc_bg, hc_ss, hsmax, ssidx, bgnum):
    hc_bg_loop, hc_ss_loop, sspar_loop, ssidx_loop, maxhs_loop, number_bg_loop \
      = ss_by_loops(edges, number, realize=False, round=True, print_test=False)
    
    # for i in range(len(ssidx)):
    #     assert np.all(ssidx[i, 0:3] == ssidx_loop[i,:]), \
    #         f"ssidx[{i}] by ndars does not match by loops"
    assert (np.all(bgnum == number_bg_loop)), "bgnum by ndars does not match by loops"
    assert (np.all(hc_ss == hc_ss_loop)), "hc_ss by ndars does not match by loops"    
    assert (np.all(hsmax == maxhs_loop)), "hsmax by ndars does not match by loops" 
    assert (np.all(hsmax == maxhs_loop)), "hsmax by ndars does not match by loops"
    if (np.all(np.isclose(hc_bg, hc_bg_loop, atol=1e-20, rtol=1e-20)) !=True):
        print("hc_bg by ndars does not match by loops, using atol=1e-20, rtol=1e-20")
    print('compare to loops test passed')

def quadratic_sum_test(hc_bg, hc_ss, hc_tt, print_test):
    test = (hc_bg**2 + hc_ss**2)
    error = (test-hc_tt**2)/hc_tt**2
    assert np.all(np.isclose(hc_tt, test, atol=2e-15, rtol=1e-15)), \
        "quadratic sum of hc_bg and hc_ss does not match hc_tt"
    if(print_test):
        print('percent error between (hc_bg^2+hc_ss^2) and hc_tt^2:', error)
        print('differences between np.sqrt((hc_bg^2+hc_ss^2)) and hc_tt:', 
              np.sqrt(test) - hc_tt)
    print('quadratic sum test passed')



def run_ndars_tests(edges,number, fobs, exname='', print_test=False, 
                     loop_comparison = True):
    '''
    Call tests for some edges, number
    Paramaters
    ----------
    edges : (4,) list of 1D arrays
        Mass, ratio, redshift, and frequency edges of bins
    number : (M, Q, Z, F) ndarray of scalars
        Number of binaries in each bin
    fobs : (F,) array of scalars
        Observed frequency bin centers
    exname : String
        Name of example (used for number plots)

    Returns
    ------
    hsamp
    hsmax
    ssidx
    bgnum
    '''
    hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum = ss_by_ndars(edges, number, realize=False, round=True)
    max_test(hsmax, hsamp)

    ssidx_test(hsmax, hsamp, ssidx, print_test)
    
    # ssnew_test(hsmax, hsamp, ssnew, print_test)

    # rounded = np.floor(number).astype(np.int64)
    # number_test(rounded, bgnum, fobs, exname, plot_test=print_test)
    
    if(loop_comparison): # optional because its faster without
        compare_to_loops_test(edges, number, hc_bg, hc_ss, hsmax, ssidx, bgnum)

    hc_tt = gws_by_ndars(edges, number, realize=False, round = True, sum=True)  
    quadratic_sum_test(hc_bg, hc_ss, hc_tt, print_test)


    return hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum

run ndars tests

In [ ]:
edges, number, fobs, exname = example2(print_test=False)
hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum = \
    run_ndars_tests(edges, number, fobs, exname, print_test=False)

In [ ]:
edges, number, fobs, exname = example3(print_test=False)
hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum = \
    run_ndars_tests(edges, number, fobs, exname, print_test=False)

In [ ]:
edges, number, fobs, exname = example4(print_test=False)
hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum = \
    run_ndars_tests(edges, number, fobs, exname, print_test=False)

### grid:

1x  |  xx  |  xx  |  xx  |  xx \
xx  |  9x  |  x3  |  xx  |  xx \
xx  |  xx  |  xx  |  x4  |  x5 

In [ ]:
grid = np.array([[[[1,0,0,0,0],[0,9,0,0,0],[0,0,0,0,0]]],
                 [[[0,0,0,0,0],[0,0,3,0,0],[0,0,0,4,5]]]])
print(grid.shape)

## 1.4 Random Other Functions

In [ ]:
def subtract_from_number(bgnum, ssidx):
    ''' 
    Inefficient way to subtract 1 from the number of all bins 
    with a loudest single source
    
    Paramaters
    -------------
    bgnum : (M, Q, Z, F) array
        number of sources in each bin, before
        single sources have been subtracted
    ssidx : (F, 4) array 
        The indices (m_idx, q_idx, and z_idx) of the parameters of the 
        loudest single source's bin, at each frequency

    Returns
    -------------
    bgnum : (M, Q, Z, F) array
        number of background sources in each bin,
        after single sources have been subtracted
    '''

    for ff in range(len(ssidx)):
        m,q,z,f = ssidx[ff]
        print(m,q,z,f)
        bgnum[m,q,z,f] -=1
    return bgnum

def max_index_at_f(grid):
    """ Get the index of the maximum value for each frequency. 
    Frequency is the 4th dimension of the 4Darray, grid.

    Parameters:
    grid : [M,Q,Z,F] array
        test grid

    Returns:
    mqz_f : (3, F) array
        Indices of max grid value at each frequency
        For F frequencies, it looks like 
        [[m1,m2,...,mF], [q1,q2,...,qF], [z1,z2,...,zF]] 
    f_mqz : (F, 3) array
        Indices of max grid value at each frequency
        For F frequencies it looks like
        [[m1,q1,z1,1], [m2,q2,z2,2], ..., [mF,qF,zF,F]]
    all_f : (4, F) array
        Indices of max grid value at each frequency
        For n frequencies, it looks like 
        [[m1,m2,...,mF], [q1,q2,...,qF], [z1,z2,...,zF], [1,2,...,F]] 
    f_all : (F, 4) array
        Indices of max grid value at each frequency
        For F frequencies it looks like
        [[m1,q1,z1,1], [m2,q2,z2,2], ..., [mF,qF,zF,F]]
    """
    mqz_f = np.empty((3,len(grid[0,0,0,:])))
    f_mqz = np.empty((len(grid[0,0,0,:]),3))
    all_f = np.empty((4, len(grid[0,0,0,:])))
    f_all = np.empty((len(grid[0,0,0,:]),4))

    for f_idx in range(len(grid[0,0,0,:])):
        m,q,z = np.unravel_index(np.argmax(grid[...,f_idx]), 
                                 grid[...,f_idx].shape)
        mqz_f[:,f_idx] = np.array([m,q,z])
        f_mqz[f_idx,:] = np.array([m,q,z])
        all_f[:, f_idx] = np.array([m,q,z,f_idx])
        f_all[f_idx,:] = np.array([m,q,z,f_idx])
    mqz_f = mqz_f.astype(int)
    f_mqz = f_mqz.astype(int)
    all_f = all_f.astype(int)
    f_all = f_all.astype(int)
    return mqz_f, f_mqz, all_f, f_all

def argwhere_at_f(grid):
    """ Get the index of the maximum value for each frequency. 
    Frequency is the 4th dimension of the 4Darray, grid.
    This should match f_all

    Parameters:
    grid : [M,Q,Z,F] array
        test grid

    Returns:
    argwhere : (F, 4) array
        Indices of max grid value at each frequency
        For F frequencies it looks like
        [[m1,q1,z1,1], [m2,q2,z2,2], ..., [mF,qF,zF,F]]
    
    """
    maxes = np.max(grid, axis=(0,1,2))
    # print(maxes)
    argwhere = np.argwhere(grid==maxes)
    # print(argwhere)
    return argwhere


def reshaped_argmax_at_f(grid, print_test=False):
    """ 
    Parameters:
    ----------
    grid : (M, Q, Z, F,) ndarray of scalars
        e.g. hsamp
    print_test : Bool
        whether or not to print values along the way
    
    Returns:
    --------
    maxidx : (3,) tuple of 1D scalar arrays
        M indicies, q indices, z indices for max value 
        in grid for each F.

    """
    shape = grid.shape
    if(print_test):
        print('old grid', grid.shape, '\n', grid)
    newshape = (shape[0]*shape[1]*shape[2], shape[3])
    grid = grid.reshape(newshape)
    if(print_test):
        print('new grid', grid.shape, '\n', grid)

    # max over all other bins at each frequency
    argmax = np.argmax(grid, axis=0)
    if(print_test):
        print('argmax', argmax)

    # unravel
    grid = grid.reshape(shape)
    if(print_test):
        print('m,q,z shape:', shape[:-1])

    # m,q,z= mqz[0][i], mqz[1][i], mqz[2][i] #, mqz[3][i] for a frequency i
    mqz = np.array(np.unravel_index(argmax, shape[:-1]))

    if(print_test):
        for i in range(len(grid[0,0,0,:])):
            m,q,z= mqz[0][i], mqz[1][i], mqz[2][i] 
            print('mqz[:,i] m,q,z,f = ', m,q,z,i)
            print('grid[m,q,z,f] =', grid[m,q,z,i])
    return mqz

def mqzf_from_mqz(mqz, print_test=False):
    f_indices = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int)
    if(print_test):
        print('f_indices', f_indices)
    mqzf = np.empty((len(f_indices),4), dtype=int)
    for i in range(len(f_indices)):
        m,q,z,f = mqz[0][i], mqz[1][i], mqz[2][i], f_indices[i]
        if(print_test):
            print('m,q,z,f', m,q,z,f)
            print('grid[m,q,z,f] =', grid[m,q,z,f])
        mqzf[i] = np.array([m,q,z,f])
    if (print_test):
        print('mqzf', mqzf.shape, '\n', mqzf)
    return mqzf
    
def faster_mqzf_from_mqz(mqz, print_test = False):
    f_indices = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int)
    mqzf = np.append(mqz, f_indices).reshape(4,len(f_indices))
    # this gives us (all m's, all q's, all z's, all f's). I like this!

    return mqzf
    # to get maxes from grid with fancy indexing
    # use grid[mqzf[0], mqzf[1], mqzf[2], mqzf[3]]

    # not certain why grid[mqzf] doesn't work
    
    # implementing this as ssnew

def ssnew(grid):
    shape = grid.shape
    newshape = (shape[0]*shape[1]*shape[2], shape[3])
    grid = grid.reshape(newshape)
   
    argmax = np.argmax(grid, axis=0) # max over all other bins at each frequency

    # unravel
    grid = grid.reshape(shape)
    # m,q,z= mqz[0][i], mqz[1][i], mqz[2][i] #, mqz[3][i] for a frequency i
    mqz = np.array(np.unravel_index(argmax, shape[:-1]))

    f_indices = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int)
    mqzf = np.append(mqz, f_indices).reshape(4,len(f_indices))
    return mqzf


In [ ]:
def test_reshaped_argmax(grid, print_test=False):
    maxes = reshaped_argmax_at_f(grid, print_test=print_test)
    print(len(maxes[0])-1)
    mqzf = mqzf_from_mqz(maxes, print_test=False)
    print(len(maxes))
    print(mqzf.shape)
    # print('mqzf', mqzf.shape, '\n', mqzf)
    for i in range(len(grid[0,0,0,:])):
        m,q,z,f = mqzf[i] #.astype(int)
        print('m,q,z,f = ', m,q,z,f, ', grid[m,q,z,f]=', grid[m,q,z,f])
test_reshaped_argmax(grid, False)


In [ ]:
def test_faster_reshaped_argmax(grid, print_test=False):
    maxes = reshaped_argmax_at_f(grid, print_test=print_test)
    print(len(maxes[0])-1)
    mqzf = mqzf_from_mqz(maxes, print_test=False)
    print(len(maxes))
    print(mqzf.shape)
    # print('mqzf', mqzf.shape, '\n', mqzf)
    for i in range(len(grid[0,0,0,:])):
        m,q,z,f = mqzf[i] #.astype(int)
        print('m,q,z,f = ', m,q,z,f, ', grid[m,q,z,f]=', grid[m,q,z,f])
test_reshaped_argmax(grid, False)

In [ ]:
def test_ssnew(grid):
    mqzf = ssnew(grid)
    print('maxes are:', grid[mqzf[0], mqzf[1], mqzf[2], mqzf[3]])
test_ssnew(grid)

In [ ]:
def test_argwhere(grid):
    # check argwhere find max indices correctly
    f_all = (max_index_at_f(grid))[-1]
    argwhere = argwhere_at_f(grid)
    assert (np.all(f_all == argwhere)), 'argwhere failing to find correct max indices'


# 2 Super Simple Example 

## 2.1 Build Model and Calculate Strains
Build Model

In [ ]:
edges, number, fobs, exname = example2(print_test=True)

Calculate Strains

In [ ]:
# Get GW Total Strain using gws_by_ndars()
hc_tt = gws_by_ndars(edges, number, realize=False, round = True, sum=True)  
# Get GW Total Strain expectation value (without sampling) using gws_by_ndars
hc_unsamp = gws_by_ndars(edges, number, realize=False, round = False, sum=True)  

Calculate Strains by ndars

In [ ]:
# Get BG and SS Strain using ss_by_ndars()
# has hsamp but not sspar
hc_bg_ndar, hc_ss_ndar, hsamp_ndar, ssidx_ndar, maxhs_ndar, bgnum_ndar\
    = run_ndars_tests(edges, number, fobs, exname, print_test=False) # round=True, realize=False 
    # = ss_by_ndars(edges, number, realize=False, round=True)

# 1 realization using ndars method
hc_bg_ndarT, hc_ss_ndarT, hsamp_ndarT, ssidx_ndarT, maxhs_ndarT, bgnum_ndarT\
    = ss_by_ndars(edges, number, realize=True)
    # = ss_by_loops(edges, number, realize=False)

# 30 realizations using ndars method
hc_bg_ndar30, hc_ss_ndar30, hsamp_ndar30, ssidx_ndar30, maxhs_ndar30, bgnum_ndar30\
    = ss_by_ndars(edges, number, realize=30)

Calculate Strains by loops

In [ ]:
# Get BG and SS Strain using loops method
# has sspar, but not hsamp
hc_bg_loop, hc_ss_loop, sspar_loop, ssidx_loop, maxhs_loop, bgnum_loop\
= ss_by_loops(edges, number, realize=False, round=True)

# 1 realization using loops method
hc_bg_loop1, hc_ss_loop1, sspar_loop1, ssidx_loop1, maxhs_loop1, bgnum_loop1\
    = ss_by_loops(edges, number, realize=1)
    # = ss_by_loops(edges, number, realize=False)

# 30 realizations using loops method
hc_bg_loop30, hc_ss_loop30, sspar_loop30, ssidx_loop30, maxhs_loop30, bgnum_loop30\
    = ss_by_loops(edges, number, realize=30)

## 2.2 Plots

### Rounded BG, SS, and TT Strain

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# total char strain (no sources subtracted)
ax.plot(xx, hc_tt, color='black', marker = 'x', lw=3, 
        ls = 'dotted', alpha=1, label='total, by loops')


# by loops:
# gwb
ax.plot(xx, hc_bg_loop, color='b', marker = 'o', lw=3,
        ls = 'dotted', alpha=.5, label='background, loops')
# loudest source per bin
ax.scatter(xx, hc_ss_loop, color='b', marker = 'o', s=100,
           edgecolor='k', alpha=.5, label='single source, loops')

# by ndars:
# gwb
ax.plot(xx, hc_bg_ndar, color='g', marker = 'o', lw=3, 
        ls = 'dotted', alpha=.5, label='background, ndars')
# loudest source per bin
ax.scatter(xx, hc_ss_ndar, color='g', marker = 'o', 
           edgecolor='k', alpha=.5, label='single source, ndars')


# ax.plot(xx, hc_bg+hc_ss, color='r', label='bg + ss', alpha=.5)
ax.plot(xx, np.sqrt(hc_bg_ndar**2+hc_ss_ndar**2), color='r', 
        label=r'$sqrt (bg^2 + ss^2)$, ndars', alpha=.5)

legend_gwb = ax.legend(bbox_to_anchor=(.95,.95), 
                       bbox_transform=fig.transFigure, loc='upper right')


# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

In [ ]:
std_bg_loop = np.std(hc_bg_loop30, axis=1)
med_bg_loop = np.median(hc_bg_loop30, axis=1)
std_ss_loop = np.std(hc_ss_loop30, axis=1)
med_ss_loop = np.median(hc_ss_loop30, axis=1)

Q75_bg_loop, Q25_bg_loop = np.percentile(hc_bg_loop30, [75 ,25], axis=1)
IQR_bg_loop = Q75_bg_loop-Q25_bg_loop
Q75_ss_loop, Q25_ss_loop = np.percentile(hc_ss_loop30, [75 ,25], axis=1)
IQR_ss_loop = Q75_ss_loop-Q25_ss_loop

std_bg_ndar = np.std(hc_bg_ndar30, axis=1)
med_bg_ndar = np.median(hc_bg_ndar30, axis=1)
std_ss_ndar = np.std(hc_ss_ndar30, axis=1)
med_ss_ndar = np.median(hc_ss_ndar30, axis=1)

Q75_bg_ndar, Q25_bg_ndar= np.percentile(hc_bg_ndar30, [75 ,25], axis=1)
IQR_bg_ndar = Q75_bg_ndar-Q25_bg_ndar
Q75_ss_ndar, Q25_ss_ndar = np.percentile(hc_ss_ndar30, [75 ,25], axis=1)
IQR_ss_ndar = Q75_ss_ndar-Q25_ss_ndar

### Stdev and IQR

In [ ]:
# IQR and Stdev 
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel=r'$h_c$ b/t 30 realizations', figsize=[10,4.5])
ax.set_title(exname+' IQR and Stdev over 30 Realizations')
xx = fobs * YR

# # plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
# yy = 1e-15 * np.power(xx, -2.0/3.0)
# ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')



ax.plot(xx, std_bg_loop, lw=4.0, alpha=0.6, color='cornflowerblue', 
        label = 'bg stdev (loop)', marker='o', ms=10, linestyle = 'dashed')
ax.plot(xx, std_ss_loop, lw=4.0, alpha=0.6, color='blue', 
        label = 'ss stdev (loop)', marker='o', ms=10, linestyle = 'solid')

ax.plot(xx, std_bg_ndar, lw=4.0, alpha=0.6, color='lightcoral', 
        label = 'bg stdev (ndar)', marker='o', ms=10, linestyle = 'dashed')
ax.plot(xx, std_ss_ndar, lw=4.0, alpha=0.6, color='r', 
        label = 'ss stdev (ndar)', marker='o', ms=10, linestyle = 'solid')

ax.plot(xx, IQR_bg_loop, lw=4.0, alpha=0.6, color='cornflowerblue', 
        label = 'bg IQR (loop)', marker = 'P', ms = 10, linestyle='dashed')
ax.plot(xx, IQR_ss_loop, lw=4.0, alpha=0.6, color='blue', 
        label = 'ss IQR (loop)', marker = 'P', ms = 10, linestyle='solid')

ax.plot(xx, IQR_bg_ndar, lw=4.0, alpha=0.6, color='lightcoral', 
        label = 'bg IQR (ndar)', marker = 'P', ms = 10, linestyle='dashed')
ax.plot(xx, IQR_ss_ndar, lw=4.0, alpha=0.6, color='r', 
        label = 'ss IQR (ndar)', marker = 'P', ms = 10, linestyle='solid')
# ax.plot(xx, IQR_loop, lw=4.0, alpha=0.6, color='blue', 
#         label = 'loop IQR', marker = 'd', ms=10, linestyle = 'dotted')
# ax.plot(xx, IQR_ndar, lw=4.0, alpha=0.6, color='green', 
#         label = 'loop ndar', marker = 'd', ms=10, linestyle = 'dotted')

legend_gwb = ax.legend(bbox_to_anchor=(.11,.93), bbox_transform=fig.transFigure, 
        loc='upper left', ncol = 4)
fig.tight_layout()

### Percentiles

In [ ]:
# Plot percentiles
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+', 25th-75th Percentiles')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# plot the model without sampling
# blue: my loop method
# red: the sam method
# ax.plot(xx, hc_loopF_sum, color='b', lw=4, ls = 'solid', label='unsampled GWB spectrum2 by loops')
# ax.plot(xx, hc_samF_sum, color='r', lw=3, ls = 'dashed', label='unsampled GWB spectrum2 by sam')

# plot just 25th and 75th percentile of bg (loop)
ax.plot(xx, Q25_bg_loop, lw=2.0, alpha=.9, color='cornflowerblue', 
        label = 'bg (loop)', marker = 'P', ms=5, linestyle = 'dashed')
ax.plot(xx, Q75_bg_loop, lw=2.0, alpha=.9, color='cornflowerblue', 
         marker = 'P', ms=5, linestyle = 'dashed')
ax.fill_between(xx, Q25_bg_loop, Q75_bg_loop, alpha=0.2, color='cornflowerblue')

# plot just the 25th and 75th percentile of bg (ndar)
ax.plot(xx, Q25_bg_ndar, lw=2.0, alpha=.9, color='lightcoral', 
        label = 'bg (ndar)', marker = 'P', ms=5, linestyle = 'dashed')
ax.plot(xx, Q75_bg_ndar, lw=2.0, alpha=.9, color='lightcoral', 
         marker = 'P', ms=5, linestyle = 'dashed')
ax.fill_between(xx, Q25_bg_ndar, Q75_bg_ndar, alpha=0.2, color='red')

# plot just 25th and 75th percentile of ss (loop)
ax.plot(xx, Q25_ss_loop, lw=2.0, alpha=.9, color='b', 
        label = 'ss  (loop)', marker = 'o', ms=5, linestyle = 'solid')
ax.plot(xx, Q75_ss_loop, lw=2.0, alpha=.9, color='b', 
         marker = 'o', ms=5, linestyle = 'solid')
ax.fill_between(xx, Q25_ss_loop, Q75_ss_loop, alpha=0.2, color='b')

# plot just 25th and 75th percentile of ss (ndar)
ax.plot(xx, Q25_ss_ndar, lw=2.0, alpha=.9, color='r', 
        label = 'ss  (ndar)', marker = 'o', ms=5, linestyle = 'solid')
ax.plot(xx, Q75_ss_ndar, lw=2.0, alpha=.9, color='r', 
         marker = 'o', ms=5, linestyle = 'solid')
ax.fill_between(xx, Q25_ss_ndar, Q75_ss_ndar, alpha=0.2, color='r')


legend_gwb = ax.legend(bbox_to_anchor=(.13,.15), bbox_transform=fig.transFigure, loc='lower left')
fig.tight_layout()

### All Ndars vs Loops

In [ ]:
def add_sample_all_to_plot(BG, SS, reals, LABEL, COLOR, rand=False):
    if(rand):
        idx = np.random.choice(BG.shape[1], nsamp, replace=False)
        ax.plot(xx, BG[:,idx], lw=2.0, alpha=0.4, color=COLOR, linestyle='dotted',
                    label=('bg (%s)' % LABEL))
        ax.scatter(xx, SS[:,idx], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.4, label=('ss (%s)' % LABEL))
    else:
        for rr in range(reals):
            if rr==reals-1:
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.4, color=COLOR, linestyle='dotted',
                        label=('bg (%s)' % LABEL))
                ax.scatter(xx, SS[:,rr], color=COLOR, marker='o', s=80,
                    edgecolor='k', alpha=0.4, label=('ss (%s)' % LABEL))

    # plot median bg of samples
    ax.plot(xx, np.median(BG, axis=1), color=COLOR, linewidth=3, 
            linestyle='solid', alpha=0.9,
            label=('bg median (%s)' % LABEL))

    # plot median ss of samples
    ax.errorbar(xx, np.median(SS, axis=1), color=COLOR,
        yerr=np.std(SS, axis=1), fmt='*', alpha=1,
        capsize=3, label=('ss median (%s)' % LABEL))


In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0)# , label = 'pure power law')

add_sample_all_to_plot(hc_bg_loop30, hc_ss_loop30, reals=5, COLOR='b', LABEL='loop')
add_sample_all_to_plot(hc_bg_ndar30, hc_ss_ndar30, reals=5, COLOR='r', LABEL='ndar')


legend_gwb = ax.legend(bbox_to_anchor=(.08,.15), fontsize=10,
                       bbox_transform=fig.transFigure, loc='lower left', ncol=3)
fig.tight_layout()

In [ ]:
def add_sample_BG_to_plot(BG, LABEL, reals, COLOR='b', rand=False):
    nsamp = BG.shape[1]
    ax.plot(xx, np.median(BG, axis=1), color=COLOR, label=LABEL)
    # plot contours at 50% and 98% confidence intervals
    for pp in [50, 98]:
        percs = pp / 2
        percs = [50 - percs, 50 + percs]
        ax.fill_between(xx, *np.percentile(BG, percs, axis=-1), alpha=0.25, color=COLOR)
    # Plot `nsamp` random spectra 
    
    if(rand):
        idx = np.random.choice(BG.shape[1], reals, replace=False)
        ax.plot(xx, BG[:, idx], lw=1.0, alpha=0.5, color=COLOR, linestyle = 'dotted')
    else: 
        for rr in range(reals):
            ax.plot(xx, BG[:, rr], lw=1.0, alpha=0.5, color=COLOR, linestyle = 'dotted')

In [ ]:
def add_sample_SS_to_plot(BG, SS, reals, incl_BG=True):
    colors = cm.rainbow(np.linspace(0,1,reals))
    for rr in range(reals):
        if rr==reals-1:
            if(incl_BG):
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.5, color=colors[rr], linestyle='solid',
                    label='background')
            ax.scatter(xx, SS[:,rr], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.5, label='single source')
        else:
            if(incl_BG):
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.5, color=colors[rr], linestyle='solid')
            ax.scatter(xx, SS[:,rr], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.5)

### Many Loops

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' loop realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_loop30, hc_ss_loop30, 10, incl_BG=True)

# plot median bg of samples
ax.plot(xx, np.median(hc_bg_loop30, axis=1), color='black', linewidth=3, 
        linestyle='solid', alpha=0.8,
        label='bg median')

# plot median ss of sampled, with errors
ax.errorbar(xx, np.median(hc_ss_loop30, axis=1), 
                yerr=np.std(hc_ss_loop30, axis=1), fmt='o', color='black', 
        capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

Many Loops Without Medians

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' loop realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_loop30, hc_ss_loop30, 10, incl_BG=True)

# # plot median bg of samples
# ax.plot(xx, np.median(hc_bg_loop30, axis=1), color='black', linewidth=3, 
#         linestyle='solid', alpha=0.8,
#         label='bg median')

# # plot median ss of sampled, with errors
# ax.errorbar(xx, np.median(hc_ss_loop30, axis=1), 
#                 yerr=np.std(hc_ss_loop30, axis=1), fmt='o', color='black', 
#         capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

### Many Ndars

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' ndar realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_ndar30, hc_ss_ndar30, 10, incl_BG=True)

# plot median bg of samples
ax.plot(xx, np.median(hc_bg_ndar30, axis=1), color='black', linewidth=3, 
        linestyle='solid', alpha=0.8,
        label='bg median')

# plot median ss of sampled, with errors
ax.errorbar(xx, np.median(hc_ss_ndar30, axis=1), 
                yerr=np.std(hc_ss_ndar30, axis=1), fmt='o', color='black', 
        capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

Many Ndars Without Medians

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' ndar realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_ndar30, hc_ss_ndar30, 10, incl_BG=True)

# # plot median bg of samples
# ax.plot(xx, np.median(hc_bg_ndar30, axis=1), color='black', linewidth=3, 
#         linestyle='solid', alpha=0.8,
#         label='bg median')

# # plot median ss of sampled, with errors
# ax.errorbar(xx, np.median(hc_ss_ndar30, axis=1), 
#                 yerr=np.std(hc_ss_ndar30, axis=1), fmt='o', color='black', 
#         capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

# 3 Medium Simple Example

## 3.1 Build Model and Calculate Strains

Build Model

In [ ]:
edges, number, fobs, exname = example3(print_test=True)

Calculate Strains

In [ ]:
# Get GW Total Strain using gws_by_ndars()
hc_tt = gws_by_ndars(edges, number, realize=False, round = True, sum=True)  
# Get GW Total Strain expectation value (without sampling) using gws_by_ndars
hc_unsamp = gws_by_ndars(edges, number, realize=False, round = False, sum=True)  

Calculate Strains by ndars

In [ ]:
# Get BG and SS Strain using ss_by_ndars()
# has hsamp but not sspar
hc_bg_ndar, hc_ss_ndar, hsamp_ndar, ssidx_ndar, maxhs_ndar, bgnum_ndar\
    = run_ndars_tests(edges, number, fobs, exname, print_test=False) # round=True, realize=False 
    # = ss_by_ndars(edges, number, realize=False, round=True)

# 1 realization using ndars method
hc_bg_ndarT, hc_ss_ndarT, hsamp_ndarT, ssidx_ndarT, maxhs_ndarT, bgnum_ndarT\
    = ss_by_ndars(edges, number, realize=True)
    # = ss_by_loops(edges, number, realize=False)

# 30 realizations using ndars method
hc_bg_ndar30, hc_ss_ndar30, hsamp_ndar30, ssidx_ndar30, maxhs_ndar30, bgnum_ndar30\
    = ss_by_ndars(edges, number, realize=30)

Calculate Strains by loops

In [ ]:
# Get BG and SS Strain using loops method
# has sspar, but not hsamp
hc_bg_loop, hc_ss_loop, sspar_loop, ssidx_loop, maxhs_loop, bgnum_loop\
= ss_by_loops(edges, number, realize=False, round=True)

# 1 realization using loops method
hc_bg_loop1, hc_ss_loop1, sspar_loop1, ssidx_loop1, maxhs_loop1, bgnum_loop1\
    = ss_by_loops(edges, number, realize=1)
    # = ss_by_loops(edges, number, realize=False)

# 30 realizations using loops method
hc_bg_loop30, hc_ss_loop30, sspar_loop30, ssidx_loop30, maxhs_loop30, bgnum_loop30\
    = ss_by_loops(edges, number, realize=30)

## 3.2 Plots

### Rounded BG, SS, and TT Strain

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# total char strain (no sources subtracted)
ax.plot(xx, hc_tt, color='black', marker = 'x', lw=3, 
        ls = 'dotted', alpha=1, label='total, by loops')


# by loops:
# gwb
ax.plot(xx, hc_bg_loop, color='b', marker = 'o', lw=3,
        ls = 'dotted', alpha=.5, label='background, loops')
# loudest source per bin
ax.scatter(xx, hc_ss_loop, color='b', marker = 'o', s=100,
           edgecolor='k', alpha=.5, label='single source, loops')

# by ndars:
# gwb
ax.plot(xx, hc_bg_ndar, color='g', marker = 'o', lw=3, 
        ls = 'dotted', alpha=.5, label='background, ndars')
# loudest source per bin
ax.scatter(xx, hc_ss_ndar, color='g', marker = 'o', 
           edgecolor='k', alpha=.5, label='single source, ndars')


# ax.plot(xx, hc_bg+hc_ss, color='r', label='bg + ss', alpha=.5)
ax.plot(xx, np.sqrt(hc_bg_ndar**2+hc_ss_ndar**2), color='r', 
        label=r'$sqrt (bg^2 + ss^2)$, ndars', alpha=.5)

legend_gwb = ax.legend(bbox_to_anchor=(.95,.95), 
                       bbox_transform=fig.transFigure, loc='upper right')


# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

In [ ]:
std_bg_loop = np.std(hc_bg_loop30, axis=1)
med_bg_loop = np.median(hc_bg_loop30, axis=1)
std_ss_loop = np.std(hc_ss_loop30, axis=1)
med_ss_loop = np.median(hc_ss_loop30, axis=1)

Q75_bg_loop, Q25_bg_loop = np.percentile(hc_bg_loop30, [75 ,25], axis=1)
IQR_bg_loop = Q75_bg_loop-Q25_bg_loop
Q75_ss_loop, Q25_ss_loop = np.percentile(hc_ss_loop30, [75 ,25], axis=1)
IQR_ss_loop = Q75_ss_loop-Q25_ss_loop

std_bg_ndar = np.std(hc_bg_ndar30, axis=1)
med_bg_ndar = np.median(hc_bg_ndar30, axis=1)
std_ss_ndar = np.std(hc_ss_ndar30, axis=1)
med_ss_ndar = np.median(hc_ss_ndar30, axis=1)

Q75_bg_ndar, Q25_bg_ndar= np.percentile(hc_bg_ndar30, [75 ,25], axis=1)
IQR_bg_ndar = Q75_bg_ndar-Q25_bg_ndar
Q75_ss_ndar, Q25_ss_ndar = np.percentile(hc_ss_ndar30, [75 ,25], axis=1)
IQR_ss_ndar = Q75_ss_ndar-Q25_ss_ndar

### Stdev and IQR

In [ ]:
# IQR and Stdev 
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel=r'$h_c$ b/t 30 realizations', figsize=[10,4.5])
ax.set_title(exname+' IQR and Stdev over 30 Realizations')
xx = fobs * YR

# # plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
# yy = 1e-15 * np.power(xx, -2.0/3.0)
# ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')



ax.plot(xx, std_bg_loop, lw=4.0, alpha=0.6, color='cornflowerblue', 
        label = 'bg stdev (loop)', marker='o', ms=10, linestyle = 'dashed')
ax.plot(xx, std_ss_loop, lw=4.0, alpha=0.6, color='blue', 
        label = 'ss stdev (loop)', marker='o', ms=10, linestyle = 'solid')

ax.plot(xx, std_bg_ndar, lw=4.0, alpha=0.6, color='lightcoral', 
        label = 'bg stdev (ndar)', marker='o', ms=10, linestyle = 'dashed')
ax.plot(xx, std_ss_ndar, lw=4.0, alpha=0.6, color='r', 
        label = 'ss stdev (ndar)', marker='o', ms=10, linestyle = 'solid')

ax.plot(xx, IQR_bg_loop, lw=4.0, alpha=0.6, color='cornflowerblue', 
        label = 'bg IQR (loop)', marker = 'P', ms = 10, linestyle='dashed')
ax.plot(xx, IQR_ss_loop, lw=4.0, alpha=0.6, color='blue', 
        label = 'ss IQR (loop)', marker = 'P', ms = 10, linestyle='solid')

ax.plot(xx, IQR_bg_ndar, lw=4.0, alpha=0.6, color='lightcoral', 
        label = 'bg IQR (ndar)', marker = 'P', ms = 10, linestyle='dashed')
ax.plot(xx, IQR_ss_ndar, lw=4.0, alpha=0.6, color='r', 
        label = 'ss IQR (ndar)', marker = 'P', ms = 10, linestyle='solid')
# ax.plot(xx, IQR_loop, lw=4.0, alpha=0.6, color='blue', 
#         label = 'loop IQR', marker = 'd', ms=10, linestyle = 'dotted')
# ax.plot(xx, IQR_ndar, lw=4.0, alpha=0.6, color='green', 
#         label = 'loop ndar', marker = 'd', ms=10, linestyle = 'dotted')

legend_gwb = ax.legend(bbox_to_anchor=(.11,.93), bbox_transform=fig.transFigure, 
        loc='upper left', ncol = 4)
fig.tight_layout()

### Percentiles

In [ ]:
# Plot percentiles
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+', 25th-75th Percentiles')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# plot the model without sampling
# blue: my loop method
# red: the sam method
# ax.plot(xx, hc_loopF_sum, color='b', lw=4, ls = 'solid', label='unsampled GWB spectrum2 by loops')
# ax.plot(xx, hc_samF_sum, color='r', lw=3, ls = 'dashed', label='unsampled GWB spectrum2 by sam')

# plot just 25th and 75th percentile of bg (loop)
ax.plot(xx, Q25_bg_loop, lw=2.0, alpha=.9, color='cornflowerblue', 
        label = 'bg (loop)', marker = 'P', ms=5, linestyle = 'dashed')
ax.plot(xx, Q75_bg_loop, lw=2.0, alpha=.9, color='cornflowerblue', 
         marker = 'P', ms=5, linestyle = 'dashed')
ax.fill_between(xx, Q25_bg_loop, Q75_bg_loop, alpha=0.2, color='cornflowerblue')

# plot just the 25th and 75th percentile of bg (ndar)
ax.plot(xx, Q25_bg_ndar, lw=2.0, alpha=.9, color='lightcoral', 
        label = 'bg (ndar)', marker = 'P', ms=5, linestyle = 'dashed')
ax.plot(xx, Q75_bg_ndar, lw=2.0, alpha=.9, color='lightcoral', 
         marker = 'P', ms=5, linestyle = 'dashed')
ax.fill_between(xx, Q25_bg_ndar, Q75_bg_ndar, alpha=0.2, color='red')

# plot just 25th and 75th percentile of ss (loop)
ax.plot(xx, Q25_ss_loop, lw=2.0, alpha=.9, color='b', 
        label = 'ss  (loop)', marker = 'o', ms=5, linestyle = 'solid')
ax.plot(xx, Q75_ss_loop, lw=2.0, alpha=.9, color='b', 
         marker = 'o', ms=5, linestyle = 'solid')
ax.fill_between(xx, Q25_ss_loop, Q75_ss_loop, alpha=0.2, color='b')

# plot just 25th and 75th percentile of ss (ndar)
ax.plot(xx, Q25_ss_ndar, lw=2.0, alpha=.9, color='r', 
        label = 'ss  (ndar)', marker = 'o', ms=5, linestyle = 'solid')
ax.plot(xx, Q75_ss_ndar, lw=2.0, alpha=.9, color='r', 
         marker = 'o', ms=5, linestyle = 'solid')
ax.fill_between(xx, Q25_ss_ndar, Q75_ss_ndar, alpha=0.2, color='r')


legend_gwb = ax.legend(bbox_to_anchor=(.13,.15), bbox_transform=fig.transFigure, loc='lower left')
fig.tight_layout()

### All Ndars vs Loops

In [ ]:
def add_sample_all_to_plot(BG, SS, reals, LABEL, COLOR, rand=False):
    if(rand):
        idx = np.random.choice(BG.shape[1], nsamp, replace=False)
        ax.plot(xx, BG[:,idx], lw=2.0, alpha=0.4, color=COLOR, linestyle='dotted',
                    label=('bg (%s)' % LABEL))
        ax.scatter(xx, SS[:,idx], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.4, label=('ss (%s)' % LABEL))
    else:
        for rr in range(reals):
            if rr==reals-1:
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.4, color=COLOR, linestyle='dotted',
                        label=('bg (%s)' % LABEL))
                ax.scatter(xx, SS[:,rr], color=COLOR, marker='o', s=80,
                    edgecolor='k', alpha=0.4, label=('ss (%s)' % LABEL))

    # plot median bg of samples
    ax.plot(xx, np.median(BG, axis=1), color=COLOR, linewidth=3, 
            linestyle='solid', alpha=0.9,
            label=('bg median (%s)' % LABEL))

    # plot median ss of samples
    ax.errorbar(xx, np.median(SS, axis=1), color=COLOR,
        yerr=np.std(SS, axis=1), fmt='*', alpha=1,
        capsize=3, label=('ss median (%s)' % LABEL))


In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0)# , label = 'pure power law')

add_sample_all_to_plot(hc_bg_loop30, hc_ss_loop30, reals=5, COLOR='b', LABEL='loop')
add_sample_all_to_plot(hc_bg_ndar30, hc_ss_ndar30, reals=5, COLOR='r', LABEL='ndar')


legend_gwb = ax.legend(bbox_to_anchor=(.08,.15), fontsize=10,
                       bbox_transform=fig.transFigure, loc='lower left', ncol=3)
fig.tight_layout()

In [ ]:
def add_sample_BG_to_plot(BG, LABEL, reals, COLOR='b', rand=False):
    nsamp = BG.shape[1]
    ax.plot(xx, np.median(BG, axis=1), color=COLOR, label=LABEL)
    # plot contours at 50% and 98% confidence intervals
    for pp in [50, 98]:
        percs = pp / 2
        percs = [50 - percs, 50 + percs]
        ax.fill_between(xx, *np.percentile(BG, percs, axis=-1), alpha=0.25, color=COLOR)
    # Plot `nsamp` random spectra 
    
    if(rand):
        idx = np.random.choice(BG.shape[1], reals, replace=False)
        ax.plot(xx, BG[:, idx], lw=1.0, alpha=0.5, color=COLOR, linestyle = 'dotted')
    else: 
        for rr in range(reals):
            ax.plot(xx, BG[:, rr], lw=1.0, alpha=0.5, color=COLOR, linestyle = 'dotted')

In [ ]:
def add_sample_SS_to_plot(BG, SS, reals, incl_BG=True):
    colors = cm.rainbow(np.linspace(0,1,reals))
    for rr in range(reals):
        if rr==reals-1:
            if(incl_BG):
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.5, color=colors[rr], linestyle='solid',
                    label='background')
            ax.scatter(xx, SS[:,rr], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.5, label='single source')
        else:
            if(incl_BG):
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.5, color=colors[rr], linestyle='solid')
            ax.scatter(xx, SS[:,rr], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.5)

### Many Loops

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' loop realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_loop30, hc_ss_loop30, 10, incl_BG=True)

# plot median bg of samples
ax.plot(xx, np.median(hc_bg_loop30, axis=1), color='black', linewidth=3, 
        linestyle='solid', alpha=0.8,
        label='bg median')

# plot median ss of sampled, with errors
ax.errorbar(xx, np.median(hc_ss_loop30, axis=1), 
                yerr=np.std(hc_ss_loop30, axis=1), fmt='o', color='black', 
        capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

Many Loops Without Medians

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' loop realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_loop30, hc_ss_loop30, 10, incl_BG=True)

# # plot median bg of samples
# ax.plot(xx, np.median(hc_bg_loop30, axis=1), color='black', linewidth=3, 
#         linestyle='solid', alpha=0.8,
#         label='bg median')

# # plot median ss of sampled, with errors
# ax.errorbar(xx, np.median(hc_ss_loop30, axis=1), 
#                 yerr=np.std(hc_ss_loop30, axis=1), fmt='o', color='black', 
#         capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

### Many Ndars

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' ndar realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_ndar30, hc_ss_ndar30, 10, incl_BG=True)

# plot median bg of samples
ax.plot(xx, np.median(hc_bg_ndar30, axis=1), color='black', linewidth=3, 
        linestyle='solid', alpha=0.8,
        label='bg median')

# plot median ss of sampled, with errors
ax.errorbar(xx, np.median(hc_ss_ndar30, axis=1), 
                yerr=np.std(hc_ss_ndar30, axis=1), fmt='o', color='black', 
        capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

Many Ndars Without Medians

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' ndar realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_ndar30, hc_ss_ndar30, 10, incl_BG=True)

# # plot median bg of samples
# ax.plot(xx, np.median(hc_bg_ndar30, axis=1), color='black', linewidth=3, 
#         linestyle='solid', alpha=0.8,
#         label='bg median')

# # plot median ss of sampled, with errors
# ax.errorbar(xx, np.median(hc_ss_ndar30, axis=1), 
#                 yerr=np.std(hc_ss_ndar30, axis=1), fmt='o', color='black', 
#         capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

# 4 Complex Example - Casting 64bit


## 4.1 Build Model and Calculate Strains
Build Model

In [ ]:
edges, number, fobs, exname = example4(print_test=True)

Calculate Strains

In [ ]:
# Get GW Total Strain using gws_by_ndars()
hc_tt = gws_by_ndars(edges, number, realize=False, round = True, sum=True)  
# Get GW Total Strain expectation value (without sampling) using gws_by_ndars
hc_unsamp = gws_by_ndars(edges, number, realize=False, round = False, sum=True)  

Calculate Strains by ndars

In [ ]:
# Get BG and SS Strain using ss_by_ndars()
# has hsamp but not sspar
hc_bg_ndar, hc_ss_ndar, hsamp_ndar, ssidx_ndar, maxhs_ndar, bgnum_ndar\
    = run_ndars_tests(edges, number, fobs, exname, print_test=False) # round=True, realize=False 
    # = ss_by_ndars(edges, number, realize=False, round=True)

# 1 realization using ndars method
hc_bg_ndarT, hc_ss_ndarT, hsamp_ndarT, ssidx_ndarT, maxhs_ndarT, bgnum_ndarT\
    = ss_by_ndars(edges, number, realize=True)
    # = ss_by_loops(edges, number, realize=False)

# 30 realizations using ndars method
hc_bg_ndar30, hc_ss_ndar30, hsamp_ndar30, ssidx_ndar30, maxhs_ndar30, bgnum_ndar30\
    = ss_by_ndars(edges, number, realize=30)

Calculate Strains by loops

In [ ]:
# Get BG and SS Strain using loops method
# has sspar, but not hsamp
hc_bg_loop, hc_ss_loop, sspar_loop, ssidx_loop, maxhs_loop, bgnum_loop\
= ss_by_loops(edges, number, realize=False, round=True)

# 1 realization using loops method
hc_bg_loop1, hc_ss_loop1, sspar_loop1, ssidx_loop1, maxhs_loop1, bgnum_loop1\
    = ss_by_loops(edges, number, realize=1)
    # = ss_by_loops(edges, number, realize=False)

# 30 realizations using loops method
hc_bg_loop30, hc_ss_loop30, sspar_loop30, ssidx_loop30, maxhs_loop30, bgnum_loop30\
    = ss_by_loops(edges, number, realize=30)

## 4.2 Plots

### Rounded BG, SS, and TT Strain

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# total char strain (no sources subtracted)
ax.plot(xx, hc_tt, color='black', marker = 'x', lw=3, 
        ls = 'dotted', alpha=1, label='total, by loops')


# by loops:
# gwb
ax.plot(xx, hc_bg_loop, color='b', marker = 'o', lw=3,
        ls = 'dotted', alpha=.5, label='background, loops')
# loudest source per bin
ax.scatter(xx, hc_ss_loop, color='b', marker = 'o', s=100,
           edgecolor='k', alpha=.5, label='single source, loops')

# by ndars:
# gwb
ax.plot(xx, hc_bg_ndar, color='g', marker = 'o', lw=3, 
        ls = 'dotted', alpha=.5, label='background, ndars')
# loudest source per bin
ax.scatter(xx, hc_ss_ndar, color='g', marker = 'o', 
           edgecolor='k', alpha=.5, label='single source, ndars')


# ax.plot(xx, hc_bg+hc_ss, color='r', label='bg + ss', alpha=.5)
ax.plot(xx, np.sqrt(hc_bg_ndar**2+hc_ss_ndar**2), color='r', 
        label=r'$sqrt (bg^2 + ss^2)$, ndars', alpha=.5)

legend_gwb = ax.legend(bbox_to_anchor=(.95,.95), 
                       bbox_transform=fig.transFigure, loc='upper right')


# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

In [ ]:
std_bg_loop = np.std(hc_bg_loop30, axis=1)
med_bg_loop = np.median(hc_bg_loop30, axis=1)
std_ss_loop = np.std(hc_ss_loop30, axis=1)
med_ss_loop = np.median(hc_ss_loop30, axis=1)

Q75_bg_loop, Q25_bg_loop = np.percentile(hc_bg_loop30, [75 ,25], axis=1)
IQR_bg_loop = Q75_bg_loop-Q25_bg_loop
Q75_ss_loop, Q25_ss_loop = np.percentile(hc_ss_loop30, [75 ,25], axis=1)
IQR_ss_loop = Q75_ss_loop-Q25_ss_loop

std_bg_ndar = np.std(hc_bg_ndar30, axis=1)
med_bg_ndar = np.median(hc_bg_ndar30, axis=1)
std_ss_ndar = np.std(hc_ss_ndar30, axis=1)
med_ss_ndar = np.median(hc_ss_ndar30, axis=1)

Q75_bg_ndar, Q25_bg_ndar= np.percentile(hc_bg_ndar30, [75 ,25], axis=1)
IQR_bg_ndar = Q75_bg_ndar-Q25_bg_ndar
Q75_ss_ndar, Q25_ss_ndar = np.percentile(hc_ss_ndar30, [75 ,25], axis=1)
IQR_ss_ndar = Q75_ss_ndar-Q25_ss_ndar

### Stdev and IQR

In [ ]:
# IQR and Stdev 
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel=r'$h_c$ b/t 30 realizations', figsize=[10,4.5])
ax.set_title(exname+' IQR and Stdev over 30 Realizations')
xx = fobs * YR

# # plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
# yy = 1e-15 * np.power(xx, -2.0/3.0)
# ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')



ax.plot(xx, std_bg_loop, lw=4.0, alpha=0.6, color='cornflowerblue', 
        label = 'bg stdev (loop)', marker='o', ms=10, linestyle = 'dashed')
ax.plot(xx, std_ss_loop, lw=4.0, alpha=0.6, color='blue', 
        label = 'ss stdev (loop)', marker='o', ms=10, linestyle = 'solid')

ax.plot(xx, std_bg_ndar, lw=4.0, alpha=0.6, color='lightcoral', 
        label = 'bg stdev (ndar)', marker='o', ms=10, linestyle = 'dashed')
ax.plot(xx, std_ss_ndar, lw=4.0, alpha=0.6, color='r', 
        label = 'ss stdev (ndar)', marker='o', ms=10, linestyle = 'solid')

ax.plot(xx, IQR_bg_loop, lw=4.0, alpha=0.6, color='cornflowerblue', 
        label = 'bg IQR (loop)', marker = 'P', ms = 10, linestyle='dashed')
ax.plot(xx, IQR_ss_loop, lw=4.0, alpha=0.6, color='blue', 
        label = 'ss IQR (loop)', marker = 'P', ms = 10, linestyle='solid')

ax.plot(xx, IQR_bg_ndar, lw=4.0, alpha=0.6, color='lightcoral', 
        label = 'bg IQR (ndar)', marker = 'P', ms = 10, linestyle='dashed')
ax.plot(xx, IQR_ss_ndar, lw=4.0, alpha=0.6, color='r', 
        label = 'ss IQR (ndar)', marker = 'P', ms = 10, linestyle='solid')
# ax.plot(xx, IQR_loop, lw=4.0, alpha=0.6, color='blue', 
#         label = 'loop IQR', marker = 'd', ms=10, linestyle = 'dotted')
# ax.plot(xx, IQR_ndar, lw=4.0, alpha=0.6, color='green', 
#         label = 'loop ndar', marker = 'd', ms=10, linestyle = 'dotted')

legend_gwb = ax.legend(bbox_to_anchor=(.11,.93), bbox_transform=fig.transFigure, 
        loc='upper left', ncol = 4)
fig.tight_layout()

### Percentiles

In [ ]:
# Plot percentiles
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+', 25th-75th Percentiles')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# plot the model without sampling
# blue: my loop method
# red: the sam method
# ax.plot(xx, hc_loopF_sum, color='b', lw=4, ls = 'solid', label='unsampled GWB spectrum2 by loops')
# ax.plot(xx, hc_samF_sum, color='r', lw=3, ls = 'dashed', label='unsampled GWB spectrum2 by sam')

# plot just 25th and 75th percentile of bg (loop)
ax.plot(xx, Q25_bg_loop, lw=2.0, alpha=.9, color='cornflowerblue', 
        label = 'bg (loop)', marker = 'P', ms=5, linestyle = 'dashed')
ax.plot(xx, Q75_bg_loop, lw=2.0, alpha=.9, color='cornflowerblue', 
         marker = 'P', ms=5, linestyle = 'dashed')
ax.fill_between(xx, Q25_bg_loop, Q75_bg_loop, alpha=0.2, color='cornflowerblue')

# plot just the 25th and 75th percentile of bg (ndar)
ax.plot(xx, Q25_bg_ndar, lw=2.0, alpha=.9, color='lightcoral', 
        label = 'bg (ndar)', marker = 'P', ms=5, linestyle = 'dashed')
ax.plot(xx, Q75_bg_ndar, lw=2.0, alpha=.9, color='lightcoral', 
         marker = 'P', ms=5, linestyle = 'dashed')
ax.fill_between(xx, Q25_bg_ndar, Q75_bg_ndar, alpha=0.2, color='red')

# plot just 25th and 75th percentile of ss (loop)
ax.plot(xx, Q25_ss_loop, lw=2.0, alpha=.9, color='b', 
        label = 'ss  (loop)', marker = 'o', ms=5, linestyle = 'solid')
ax.plot(xx, Q75_ss_loop, lw=2.0, alpha=.9, color='b', 
         marker = 'o', ms=5, linestyle = 'solid')
ax.fill_between(xx, Q25_ss_loop, Q75_ss_loop, alpha=0.2, color='b')

# plot just 25th and 75th percentile of ss (ndar)
ax.plot(xx, Q25_ss_ndar, lw=2.0, alpha=.9, color='r', 
        label = 'ss  (ndar)', marker = 'o', ms=5, linestyle = 'solid')
ax.plot(xx, Q75_ss_ndar, lw=2.0, alpha=.9, color='r', 
         marker = 'o', ms=5, linestyle = 'solid')
ax.fill_between(xx, Q25_ss_ndar, Q75_ss_ndar, alpha=0.2, color='r')


legend_gwb = ax.legend(bbox_to_anchor=(.13,.15), bbox_transform=fig.transFigure, loc='lower left')
fig.tight_layout()

### All Ndars vs Loops

In [ ]:
def add_sample_all_to_plot(BG, SS, reals, LABEL, COLOR, rand=False):
    if(rand):
        idx = np.random.choice(BG.shape[1], nsamp, replace=False)
        ax.plot(xx, BG[:,idx], lw=2.0, alpha=0.4, color=COLOR, linestyle='dotted',
                    label=('bg (%s)' % LABEL))
        ax.scatter(xx, SS[:,idx], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.4, label=('ss (%s)' % LABEL))
    else:
        for rr in range(reals):
            if rr==reals-1:
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.4, color=COLOR, linestyle='dotted',
                        label=('bg (%s)' % LABEL))
                ax.scatter(xx, SS[:,rr], color=COLOR, marker='o', s=80,
                    edgecolor='k', alpha=0.4, label=('ss (%s)' % LABEL))

    # plot median bg of samples
    ax.plot(xx, np.median(BG, axis=1), color=COLOR, linewidth=3, 
            linestyle='solid', alpha=0.9,
            label=('bg median (%s)' % LABEL))

    # plot median ss of samples
    ax.errorbar(xx, np.median(SS, axis=1), color=COLOR,
        yerr=np.std(SS, axis=1), fmt='*', alpha=1,
        capsize=3, label=('ss median (%s)' % LABEL))


In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0)# , label = 'pure power law')

add_sample_all_to_plot(hc_bg_loop30, hc_ss_loop30, reals=5, COLOR='b', LABEL='loop')
add_sample_all_to_plot(hc_bg_ndar30, hc_ss_ndar30, reals=5, COLOR='r', LABEL='ndar')


legend_gwb = ax.legend(bbox_to_anchor=(.08,.15), fontsize=10,
                       bbox_transform=fig.transFigure, loc='lower left', ncol=3)
fig.tight_layout()

In [ ]:
def add_sample_BG_to_plot(BG, LABEL, reals, COLOR='b', rand=False):
    nsamp = BG.shape[1]
    ax.plot(xx, np.median(BG, axis=1), color=COLOR, label=LABEL)
    # plot contours at 50% and 98% confidence intervals
    for pp in [50, 98]:
        percs = pp / 2
        percs = [50 - percs, 50 + percs]
        ax.fill_between(xx, *np.percentile(BG, percs, axis=-1), alpha=0.25, color=COLOR)
    # Plot `nsamp` random spectra 
    
    if(rand):
        idx = np.random.choice(BG.shape[1], reals, replace=False)
        ax.plot(xx, BG[:, idx], lw=1.0, alpha=0.5, color=COLOR, linestyle = 'dotted')
    else: 
        for rr in range(reals):
            ax.plot(xx, BG[:, rr], lw=1.0, alpha=0.5, color=COLOR, linestyle = 'dotted')

In [ ]:
def add_sample_SS_to_plot(BG, SS, reals, incl_BG=True):
    colors = cm.rainbow(np.linspace(0,1,reals))
    for rr in range(reals):
        if rr==reals-1:
            if(incl_BG):
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.5, color=colors[rr], linestyle='solid',
                    label='background')
            ax.scatter(xx, SS[:,rr], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.5, label='single source')
        else:
            if(incl_BG):
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.5, color=colors[rr], linestyle='solid')
            ax.scatter(xx, SS[:,rr], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.5)

### Many Loops

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' loop realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_loop30, hc_ss_loop30, 10, incl_BG=True)

# plot median bg of samples
ax.plot(xx, np.median(hc_bg_loop30, axis=1), color='black', linewidth=3, 
        linestyle='solid', alpha=0.8,
        label='bg median')

# plot median ss of sampled, with errors
ax.errorbar(xx, np.median(hc_ss_loop30, axis=1), 
                yerr=np.std(hc_ss_loop30, axis=1), fmt='o', color='black', 
        capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

Many Loops Without Medians

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' loop realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_loop30, hc_ss_loop30, 10, incl_BG=True)

# # plot median bg of samples
# ax.plot(xx, np.median(hc_bg_loop30, axis=1), color='black', linewidth=3, 
#         linestyle='solid', alpha=0.8,
#         label='bg median')

# # plot median ss of sampled, with errors
# ax.errorbar(xx, np.median(hc_ss_loop30, axis=1), 
#                 yerr=np.std(hc_ss_loop30, axis=1), fmt='o', color='black', 
#         capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

### Many Ndars

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' ndar realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_ndar30, hc_ss_ndar30, 10, incl_BG=True)

# plot median bg of samples
ax.plot(xx, np.median(hc_bg_ndar30, axis=1), color='black', linewidth=3, 
        linestyle='solid', alpha=0.8,
        label='bg median')

# plot median ss of sampled, with errors
ax.errorbar(xx, np.median(hc_ss_ndar30, axis=1), 
                yerr=np.std(hc_ss_ndar30, axis=1), fmt='o', color='black', 
        capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

Many Ndars Without Medians

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' ndar realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_ndar30, hc_ss_ndar30, 10, incl_BG=True)

# # plot median bg of samples
# ax.plot(xx, np.median(hc_bg_ndar30, axis=1), color='black', linewidth=3, 
#         linestyle='solid', alpha=0.8,
#         label='bg median')

# # plot median ss of sampled, with errors
# ax.errorbar(xx, np.median(hc_ss_ndar30, axis=1), 
#                 yerr=np.std(hc_ss_ndar30, axis=1), fmt='o', color='black', 
#         capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

# 5 SAM Default Example - Casting 64bit


## 5.1 Build Model and Calculate Strains
Build Model

In [ ]:
edges, number, fobs, exname = example5(print_test=True)

In [ ]:
print(bgnum_ndar30.shape)

Calculate Strains

In [ ]:
# Get GW Total Strain using gws_by_ndars()
hc_tt = gws_by_ndars(edges, number, realize=False, round = True, sum=True)  
# Get GW Total Strain expectation value (without sampling) using gws_by_ndars
hc_unsamp = gws_by_ndars(edges, number, realize=False, round = False, sum=True)  

Calculate Strains by ndars 

(~20s to run)

In [ ]:
# Get BG and SS Strain using ss_by_ndars()
# has hsamp but not sspar
hc_bg_ndar, hc_ss_ndar, hsamp_ndar, ssidx_ndar, maxhs_ndar, bgnum_ndar\
    = ss_by_ndars(edges, number, realize=False, round=True)
    # = run_ndars_tests(edges, number, fobs, exname, print_test=False) # round=True, realize=False 
    
# 1 realization using ndars method
hc_bg_ndarT, hc_ss_ndarT, hsamp_ndarT, ssidx_ndarT, maxhs_ndarT, bgnum_ndarT\
    = ss_by_ndars(edges, number, realize=True)
    # = ss_by_loops(edges, number, realize=False)

# 30 realizations using ndars method
hc_bg_ndar30, hc_ss_ndar30, hsamp_ndar30, ssidx_ndar30, maxhs_ndar30, bgnum_ndar30\
    = ss_by_ndars(edges, number, realize=30)

Calculate Strains by loops

(~15min to run)

In [ ]:
# Get BG and SS Strain using loops method
# has sspar, but not hsamp
hc_bg_loop, hc_ss_loop, sspar_loop, ssidx_loop, maxhs_loop, bgnum_loop\
= ss_by_loops(edges, number, realize=False, round=True)

# 1 realization using loops method
hc_bg_loop1, hc_ss_loop1, sspar_loop1, ssidx_loop1, maxhs_loop1, bgnum_loop1\
    = ss_by_loops(edges, number, realize=1)
    # = ss_by_loops(edges, number, realize=False)

# 30 realizations using loops method
hc_bg_loop30, hc_ss_loop30, sspar_loop30, ssidx_loop30, maxhs_loop30, bgnum_loop30\
    = ss_by_loops(edges, number, realize=30)
np.savez('/Users/emigardiner/GWs/holodeck/ecg-notebooks\
    /single_sources/ss-dev/examp5_loop.npz', 
        hc_bg_loop30=hc_bg_loop30, 
        hc_ss_loop30=hc_ss_loop30, 
        sspar_loop30=sspar_loop30, 
        ssidx_loop30=ssidx_loop30, 
        maxhs_loop30=maxhs_loop30, 
        bgnum_loop30=bgnum_loop30)
hc_bg_examp5, hc_ss_examp5, sspar_examp5, ssidx_examp5, maxhs_examp5, bgnum_examp5\
 = hc_bg_loop30, hc_ss_loop30, sspar_loop30, ssidx_loop30, maxhs_loop30, bgnum_loop30
# alternatively, load from npz file

Timeit

In [ ]:
%timeit hc_bg_ndar30, hc_ss_ndar30, hsamp_ndar30, ssidx_ndar30, maxhs_ndar30, bgnum_ndar30\
    = ss_by_ndars(edges, number, realize=30)

In [ ]:
%time hc_bg_loop30, hc_ss_loop30, sspar_loop30, ssidx_loop30, maxhs_loop30, bgnum_loop30\
    = ss_by_loops(edges, number, realize=30)

In [ ]:
%timeit hc_bg_loop30, hc_ss_loop30, sspar_loop30, ssidx_loop30, maxhs_loop30, bgnum_loop30\
    = ss_by_loops(edges, number, realize=30)

## 5.2 Plots

### Rounded BG, SS, and TT Strain

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# total char strain (no sources subtracted)
ax.plot(xx, hc_tt, color='black', marker = 'x', lw=3, 
        ls = 'dotted', alpha=1, label='total, by loops')


# by loops:
# gwb
ax.plot(xx, hc_bg_loop, color='b', marker = 'o', lw=3,
        ls = 'dotted', alpha=.5, label='background, loops')
# loudest source per bin
ax.scatter(xx, hc_ss_loop, color='b', marker = 'o', s=100,
           edgecolor='k', alpha=.5, label='single source, loops')

# by ndars:
# gwb
ax.plot(xx, hc_bg_ndar, color='g', marker = 'o', lw=3, 
        ls = 'dotted', alpha=.5, label='background, ndars')
# loudest source per bin
ax.scatter(xx, hc_ss_ndar, color='g', marker = 'o', 
           edgecolor='k', alpha=.5, label='single source, ndars')


# ax.plot(xx, hc_bg+hc_ss, color='r', label='bg + ss', alpha=.5)
ax.plot(xx, np.sqrt(hc_bg_ndar**2+hc_ss_ndar**2), color='r', 
        label=r'$sqrt (bg^2 + ss^2)$, ndars', alpha=.5)

legend_gwb = ax.legend(bbox_to_anchor=(.95,.95), 
                       bbox_transform=fig.transFigure, loc='upper right')


# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

In [ ]:
std_bg_loop = np.std(hc_bg_loop30, axis=1)
med_bg_loop = np.median(hc_bg_loop30, axis=1)
std_ss_loop = np.std(hc_ss_loop30, axis=1)
med_ss_loop = np.median(hc_ss_loop30, axis=1)

Q75_bg_loop, Q25_bg_loop = np.percentile(hc_bg_loop30, [75 ,25], axis=1)
IQR_bg_loop = Q75_bg_loop-Q25_bg_loop
Q75_ss_loop, Q25_ss_loop = np.percentile(hc_ss_loop30, [75 ,25], axis=1)
IQR_ss_loop = Q75_ss_loop-Q25_ss_loop

std_bg_ndar = np.std(hc_bg_ndar30, axis=1)
med_bg_ndar = np.median(hc_bg_ndar30, axis=1)
std_ss_ndar = np.std(hc_ss_ndar30, axis=1)
med_ss_ndar = np.median(hc_ss_ndar30, axis=1)

Q75_bg_ndar, Q25_bg_ndar= np.percentile(hc_bg_ndar30, [75 ,25], axis=1)
IQR_bg_ndar = Q75_bg_ndar-Q25_bg_ndar
Q75_ss_ndar, Q25_ss_ndar = np.percentile(hc_ss_ndar30, [75 ,25], axis=1)
IQR_ss_ndar = Q75_ss_ndar-Q25_ss_ndar

### Stdev and IQR

In [ ]:
# IQR and Stdev 
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel=r'$h_c$ b/t 30 realizations', figsize=[10,4.5])
ax.set_title(exname+' IQR and Stdev over 30 Realizations')
xx = fobs * YR

# # plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
# yy = 1e-15 * np.power(xx, -2.0/3.0)
# ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')



ax.plot(xx, std_bg_loop, lw=4.0, alpha=0.6, color='cornflowerblue', 
        label = 'bg stdev (loop)', marker='o', ms=10, linestyle = 'dashed')
ax.plot(xx, std_ss_loop, lw=4.0, alpha=0.6, color='blue', 
        label = 'ss stdev (loop)', marker='o', ms=10, linestyle = 'solid')

ax.plot(xx, std_bg_ndar, lw=4.0, alpha=0.6, color='lightcoral', 
        label = 'bg stdev (ndar)', marker='o', ms=10, linestyle = 'dashed')
ax.plot(xx, std_ss_ndar, lw=4.0, alpha=0.6, color='r', 
        label = 'ss stdev (ndar)', marker='o', ms=10, linestyle = 'solid')

ax.plot(xx, IQR_bg_loop, lw=4.0, alpha=0.6, color='cornflowerblue', 
        label = 'bg IQR (loop)', marker = 'P', ms = 10, linestyle='dashed')
ax.plot(xx, IQR_ss_loop, lw=4.0, alpha=0.6, color='blue', 
        label = 'ss IQR (loop)', marker = 'P', ms = 10, linestyle='solid')

ax.plot(xx, IQR_bg_ndar, lw=4.0, alpha=0.6, color='lightcoral', 
        label = 'bg IQR (ndar)', marker = 'P', ms = 10, linestyle='dashed')
ax.plot(xx, IQR_ss_ndar, lw=4.0, alpha=0.6, color='r', 
        label = 'ss IQR (ndar)', marker = 'P', ms = 10, linestyle='solid')
# ax.plot(xx, IQR_loop, lw=4.0, alpha=0.6, color='blue', 
#         label = 'loop IQR', marker = 'd', ms=10, linestyle = 'dotted')
# ax.plot(xx, IQR_ndar, lw=4.0, alpha=0.6, color='green', 
#         label = 'loop ndar', marker = 'd', ms=10, linestyle = 'dotted')

legend_gwb = ax.legend(bbox_to_anchor=(.11,.93), bbox_transform=fig.transFigure, 
        loc='upper left', ncol = 4)
fig.tight_layout()

### Percentiles

In [ ]:
# Plot percentiles
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+', 25th-75th Percentiles')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# plot the model without sampling
# blue: my loop method
# red: the sam method
# ax.plot(xx, hc_loopF_sum, color='b', lw=4, ls = 'solid', label='unsampled GWB spectrum2 by loops')
# ax.plot(xx, hc_samF_sum, color='r', lw=3, ls = 'dashed', label='unsampled GWB spectrum2 by sam')

# plot just 25th and 75th percentile of bg (loop)
ax.plot(xx, Q25_bg_loop, lw=2.0, alpha=.9, color='cornflowerblue', 
        label = 'bg (loop)', marker = 'P', ms=5, linestyle = 'dashed')
ax.plot(xx, Q75_bg_loop, lw=2.0, alpha=.9, color='cornflowerblue', 
         marker = 'P', ms=5, linestyle = 'dashed')
ax.fill_between(xx, Q25_bg_loop, Q75_bg_loop, alpha=0.2, color='cornflowerblue')

# plot just the 25th and 75th percentile of bg (ndar)
ax.plot(xx, Q25_bg_ndar, lw=2.0, alpha=.9, color='lightcoral', 
        label = 'bg (ndar)', marker = 'P', ms=5, linestyle = 'dashed')
ax.plot(xx, Q75_bg_ndar, lw=2.0, alpha=.9, color='lightcoral', 
         marker = 'P', ms=5, linestyle = 'dashed')
ax.fill_between(xx, Q25_bg_ndar, Q75_bg_ndar, alpha=0.2, color='red')

# plot just 25th and 75th percentile of ss (loop)
ax.plot(xx, Q25_ss_loop, lw=2.0, alpha=.9, color='b', 
        label = 'ss  (loop)', marker = 'o', ms=5, linestyle = 'solid')
ax.plot(xx, Q75_ss_loop, lw=2.0, alpha=.9, color='b', 
         marker = 'o', ms=5, linestyle = 'solid')
ax.fill_between(xx, Q25_ss_loop, Q75_ss_loop, alpha=0.2, color='b')

# plot just 25th and 75th percentile of ss (ndar)
ax.plot(xx, Q25_ss_ndar, lw=2.0, alpha=.9, color='r', 
        label = 'ss  (ndar)', marker = 'o', ms=5, linestyle = 'solid')
ax.plot(xx, Q75_ss_ndar, lw=2.0, alpha=.9, color='r', 
         marker = 'o', ms=5, linestyle = 'solid')
ax.fill_between(xx, Q25_ss_ndar, Q75_ss_ndar, alpha=0.2, color='r')


legend_gwb = ax.legend(bbox_to_anchor=(.13,.15), bbox_transform=fig.transFigure, loc='lower left')
fig.tight_layout()

### All Ndars vs Loops

In [ ]:
def add_sample_all_to_plot(BG, SS, reals, LABEL, COLOR, rand=False):
    if(rand):
        idx = np.random.choice(BG.shape[1], nsamp, replace=False)
        ax.plot(xx, BG[:,idx], lw=2.0, alpha=0.4, color=COLOR, linestyle='dotted',
                    label=('bg (%s)' % LABEL))
        ax.scatter(xx, SS[:,idx], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.4, label=('ss (%s)' % LABEL))
    else:
        for rr in range(reals):
            if rr==reals-1:
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.4, color=COLOR, linestyle='dotted',
                        label=('bg (%s)' % LABEL))
                ax.scatter(xx, SS[:,rr], color=COLOR, marker='o', s=80,
                    edgecolor='k', alpha=0.4, label=('ss (%s)' % LABEL))

    # plot median bg of samples
    ax.plot(xx, np.median(BG, axis=1), color=COLOR, linewidth=3, 
            linestyle='solid', alpha=0.9,
            label=('bg median (%s)' % LABEL))

    # plot median ss of samples
    ax.errorbar(xx, np.median(SS, axis=1), color=COLOR,
        yerr=np.std(SS, axis=1), fmt='*', alpha=1,
        capsize=3, label=('ss median (%s)' % LABEL))


In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname)
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0)# , label = 'pure power law')

add_sample_all_to_plot(hc_bg_loop30, hc_ss_loop30, reals=5, COLOR='b', LABEL='loop')
add_sample_all_to_plot(hc_bg_ndar30, hc_ss_ndar30, reals=5, COLOR='r', LABEL='ndar')


legend_gwb = ax.legend(bbox_to_anchor=(.08,.15), fontsize=10,
                       bbox_transform=fig.transFigure, loc='lower left', ncol=3)
fig.tight_layout()

In [ ]:
def add_sample_BG_to_plot(BG, LABEL, reals, COLOR='b', rand=False):
    nsamp = BG.shape[1]
    ax.plot(xx, np.median(BG, axis=1), color=COLOR, label=LABEL)
    # plot contours at 50% and 98% confidence intervals
    for pp in [50, 98]:
        percs = pp / 2
        percs = [50 - percs, 50 + percs]
        ax.fill_between(xx, *np.percentile(BG, percs, axis=-1), alpha=0.25, color=COLOR)
    # Plot `nsamp` random spectra 
    
    if(rand):
        idx = np.random.choice(BG.shape[1], reals, replace=False)
        ax.plot(xx, BG[:, idx], lw=1.0, alpha=0.5, color=COLOR, linestyle = 'dotted')
    else: 
        for rr in range(reals):
            ax.plot(xx, BG[:, rr], lw=1.0, alpha=0.5, color=COLOR, linestyle = 'dotted')

In [ ]:
def add_sample_SS_to_plot(BG, SS, reals, incl_BG=True):
    colors = cm.rainbow(np.linspace(0,1,reals))
    for rr in range(reals):
        if rr==reals-1:
            if(incl_BG):
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.5, color=colors[rr], linestyle='solid',
                    label='background')
            ax.scatter(xx, SS[:,rr], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.5, label='single source')
        else:
            if(incl_BG):
                ax.plot(xx, BG[:,rr], lw=2.0, alpha=0.5, color=colors[rr], linestyle='solid')
            ax.scatter(xx, SS[:,rr], color=colors[rr], marker='o', s=80,
                edgecolor='k', alpha=0.5)

### Many Loops

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' loop realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_loop30, hc_ss_loop30, 10, incl_BG=True)

# plot median bg of samples
ax.plot(xx, np.median(hc_bg_loop30, axis=1), color='black', linewidth=3, 
        linestyle='solid', alpha=0.8,
        label='bg median')

# plot median ss of sampled, with errors
ax.errorbar(xx, np.median(hc_ss_loop30, axis=1), 
                yerr=np.std(hc_ss_loop30, axis=1), fmt='o', color='black', 
        capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

Many Loops Without Medians

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' loop realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_loop30, hc_ss_loop30, 10, incl_BG=True)

# # plot median bg of samples
# ax.plot(xx, np.median(hc_bg_loop30, axis=1), color='black', linewidth=3, 
#         linestyle='solid', alpha=0.8,
#         label='bg median')

# # plot median ss of sampled, with errors
# ax.errorbar(xx, np.median(hc_ss_loop30, axis=1), 
#                 yerr=np.std(hc_ss_loop30, axis=1), fmt='o', color='black', 
#         capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

### Many Ndars

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' ndar realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_ndar30, hc_ss_ndar30, 10, incl_BG=True)

# plot median bg of samples
ax.plot(xx, np.median(hc_bg_ndar30, axis=1), color='black', linewidth=3, 
        linestyle='solid', alpha=0.8,
        label='bg median')

# plot median ss of sampled, with errors
ax.errorbar(xx, np.median(hc_ss_ndar30, axis=1), 
                yerr=np.std(hc_ss_ndar30, axis=1), fmt='o', color='black', 
        capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()

Many Ndars Without Medians

In [ ]:
# quick plot
fig, ax = plot.figax(xlabel='Frequency $f_\mathrm{obs}$ [1/yr]', 
                    ylabel='Characteristic Strain $h_c$', figsize=[10,4.5])
ax.set_title(exname+' ndar realizations')
xx = fobs * YR

# plot a reference, pure power-law  strain spectrum:   h_c(f) = 1e-15 * (f * yr) ^ -2/3
yy = 1e-15 * np.power(xx, -2.0/3.0)
ax.plot(xx, yy, 'k--', alpha=0.25, lw=2.0, label = 'pure power law')

# # unsampled/expectation value of total strain (without single source subtraction)
# ax.plot(xx, hc_unsamp, color='b',  lw=2,
#         ls = 'dashdot', alpha=1, label='total, without sampling')

# plot the model with Poisson sampling
add_sample_SS_to_plot(hc_bg_ndar30, hc_ss_ndar30, 10, incl_BG=True)

# # plot median bg of samples
# ax.plot(xx, np.median(hc_bg_ndar30, axis=1), color='black', linewidth=3, 
#         linestyle='solid', alpha=0.8,
#         label='bg median')

# # plot median ss of sampled, with errors
# ax.errorbar(xx, np.median(hc_ss_ndar30, axis=1), 
#                 yerr=np.std(hc_ss_ndar30, axis=1), fmt='o', color='black', 
#         capsize=3, label='ss median')


legend_gwb = ax.legend(bbox_to_anchor=(.1,.15), 
                       bbox_transform=fig.transFigure, loc='lower left',
                       fontsize=8)

# ax.set_ylim(1e-16, 3e-15)
fig.tight_layout()